<a href="https://colab.research.google.com/github/krishmishraghub/Log-Analyser/blob/main/Log_Analyser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
Develop a Python-based log analysis and incident correlation system that can parse various log formats, normalize log entries, detect anomalies, correlate related log events into incidents, assign severity, and generate structured reports in CSV and JSON formats. Additionally, implement a command-line interface for user interaction and visualize incident timelines and error trends using `matplotlib`.

## Design Data Structures

### Subtask:
Define Python classes or data structures for `NormalizedLogEntry` and `Incident` to store extracted log data and correlated incident information.


**Reasoning**:
To define the Python classes as requested by the instructions, I will create a code block that includes the definitions for `NormalizedLogEntry` and `Incident`.



In [ ]:
from datetime import datetime

class NormalizedLogEntry:
    """Represents a normalized log entry."""
    def __init__(self, timestamp: datetime, log_level: str, source: str, message: str):
        self.timestamp = timestamp
        self.log_level = log_level
        self.source = source
        self.message = message

    def __repr__(self):
        return f"NormalizedLogEntry(timestamp='{self.timestamp}', log_level='{self.log_level}', source='{self.source}', message='{self.message[:50]}...')"

class Incident:
    """Represents a correlated incident."""
    def __init__(
        self,
        start_time: datetime,
        end_time: datetime,
        affected_hosts_services: list,
        total_events: int,
        severity: str,
        probable_root_cause: str
    ):
        self.start_time = start_time
        self.end_time = end_time
        self.affected_hosts_services = affected_hosts_services
        self.total_events = total_events
        self.severity = severity
        self.probable_root_cause = probable_root_cause

    def __repr__(self):
        return (
            f"Incident(start_time='{self.start_time}', end_time='{self.end_time}', "
            f"affected_hosts_services={self.affected_hosts_services}, total_events={self.total_events}, "
            f"severity='{self.severity}', probable_root_cause='{self.probable_root_cause[:50]}...')"
        )

print("Defined NormalizedLogEntry and Incident classes.")

Defined NormalizedLogEntry and Incident classes.


## Implement Log Parser

### Subtask:
Develop a `LogParser` class capable of parsing various log formats (e.g., common patterns for Linux, Windows, and application logs) using regular expressions or other parsing techniques. This class should extract `timestamp`, `log_level`, `source/host`, and `message` from raw log lines. It should be extensible to add new log formats easily.


**Reasoning**:
I need to define the `LogParser` class with its constructor, `parse_log_entry`, and `add_format` methods, adhering to the specified requirements for parsing various log formats and creating `NormalizedLogEntry` objects. This will involve using regular expressions and handling datetime conversions.



In [ ]:
import re
from datetime import datetime

# Assuming NormalizedLogEntry class is already defined from a previous step
# from datetime import datetime
# class NormalizedLogEntry:
#     """Represents a normalized log entry."""
#     def __init__(self, timestamp: datetime, log_level: str, source: str, message: str):
#         self.timestamp = timestamp
#         self.log_level = log_level
#         self.source = source
#         self.message = message

#     def __repr__(self):
#         return f"NormalizedLogEntry(timestamp='{self.timestamp}', log_level='{self.log_level}', source='{self.source}', message='{self.message[:50]}...')"


class LogParser:
    """Parses raw log lines into NormalizedLogEntry objects based on configured formats."""

    def __init__(self, log_formats: list = None):
        """
        Initializes the LogParser with a list of log format configurations.

        Args:
            log_formats (list): A list of dictionaries, where each dictionary
                                 defines a log format. Each dictionary should
                                 have 'regex' (str) and 'mapping' (dict)
                                 keys. The 'mapping' keys should be regex group
                                 names and values should be NormalizedLogEntry
                                 attribute names (e.g., 'level': 'log_level').
                                 It can also include 'timestamp_format' (str)
                                 for datetime parsing.
        """
        self.log_formats = log_formats if log_formats is not None else []

    def add_format(self, regex_pattern: str, mapping: dict, timestamp_format: str = None):
        """
        Adds a new log format configuration to the parser.

        Args:
            regex_pattern (str): The regular expression pattern for the log format.
            mapping (dict): A dictionary mapping regex group names to
                            NormalizedLogEntry attribute names.
            timestamp_format (str, optional): The datetime format string for parsing
                                              the timestamp. If None, assumes ISO format
                                              or attempts common formats.
        """
        self.log_formats.append({
            'regex': re.compile(regex_pattern),
            'mapping': mapping,
            'timestamp_format': timestamp_format
        })
        print(f"Added log format: {regex_pattern}")

    def parse_log_entry(self, log_line: str) -> NormalizedLogEntry or None:
        """
        Parses a single raw log line into a NormalizedLogEntry object.

        Args:
            log_line (str): The raw log line to parse.

        Returns:
            NormalizedLogEntry or None: The parsed log entry if successful, otherwise None.
        """
        for fmt in self.log_formats:
            match = fmt['regex'].match(log_line)
            if match:
                extracted_data = match.groupdict()
                parsed_attrs = {}

                # Map extracted data to NormalizedLogEntry attributes
                for group_name, attr_name in fmt['mapping'].items():
                    if group_name in extracted_data:
                        parsed_attrs[attr_name] = extracted_data[group_name]

                # Handle timestamp conversion
                timestamp_str = parsed_attrs.get('timestamp')
                if timestamp_str:
                    try:
                        if fmt['timestamp_format']:
                            parsed_attrs['timestamp'] = datetime.strptime(timestamp_str, fmt['timestamp_format'])
                        else:
                            # Attempt common formats if specific format not provided
                            try:
                                parsed_attrs['timestamp'] = datetime.fromisoformat(timestamp_str)
                            except ValueError:
                                # Fallback for other common formats, add more if necessary
                                if 'T' in timestamp_str and '+' in timestamp_str:
                                    # Example: 2023-10-27T10:00:00.123456+00:00
                                    # Need to handle microseconds and timezone carefully
                                    parts = timestamp_str.split('+')
                                    timestamp_no_tz = parts[0]
                                    # Trim microseconds if too many digits for strptime
                                    if '.' in timestamp_no_tz:
                                        pre_dot, post_dot = timestamp_no_tz.split('.')
                                        if len(post_dot) > 6:
                                            timestamp_no_tz = f"{pre_dot}.{post_dot[:6]}"
                                    parsed_attrs['timestamp'] = datetime.strptime(timestamp_no_tz, '%Y-%m-%dT%H:%M:%S.%f')
                                else:
                                    parsed_attrs['timestamp'] = datetime.strptime(timestamp_str, '%Y-%m-%d %H:%M:%S')
                    except ValueError as e:
                        print(f"Warning: Could not parse timestamp '{timestamp_str}' with format '{fmt.get('timestamp_format', 'auto')}': {e}")
                        return None # Or raise custom exception
                else:
                    print("Warning: Timestamp not found in parsed log entry.")
                    return None

                # Ensure all required NormalizedLogEntry fields are present, providing defaults if necessary
                # Or return None if crucial fields are missing
                timestamp = parsed_attrs.get('timestamp')
                log_level = parsed_attrs.get('log_level', 'UNKNOWN')
                source = parsed_attrs.get('source', 'UNKNOWN')
                message = parsed_attrs.get('message', log_line)

                if timestamp is None: # Crucial field missing due to parsing error
                    return None

                return NormalizedLogEntry(timestamp, log_level, source, message)

        print(f"No matching format found for log line: {log_line[:100]}...")
        return None

print("Defined LogParser class.")

Defined LogParser class.


## Develop Log Normalizer

### Subtask:
Create a `LogNormalizer` component that takes raw parsed log data from different formats and transforms them into the uniform `NormalizedLogEntry` structure defined earlier. This ensures consistency across diverse log sources.


**Reasoning**:
I need to define the `LogNormalizer` class with a `normalize_logs` method that utilizes the previously defined `LogParser` to process raw log lines into `NormalizedLogEntry` objects. I will also add a print statement to confirm the class definition.



In [ ]:
class LogNormalizer:
    """Normalizes raw log lines into a list of NormalizedLogEntry objects using a LogParser."""

    def __init__(self):
        """Initializes the LogNormalizer."""
        print("LogNormalizer initialized.")

    def normalize_logs(self, log_parser: LogParser, raw_log_lines: list) -> list:
        """
        Parses and normalizes a list of raw log lines using the provided LogParser.

        Args:
            log_parser (LogParser): An instance of the LogParser class.
            raw_log_lines (list): A list of raw log line strings.

        Returns:
            list: A list of NormalizedLogEntry objects that were successfully parsed.
        """
        normalized_entries = []
        for line in raw_log_lines:
            entry = log_parser.parse_log_entry(line)
            if entry:
                normalized_entries.append(entry)
        print(f"Normalized {len(normalized_entries)} out of {len(raw_log_lines)} log entries.")
        return normalized_entries

print("Defined LogNormalizer class.")

Defined LogNormalizer class.


## Implement Anomaly Detector

### Subtask:
Design and implement an `AnomalyDetector` class that can identify predefined abnormal events such as errors, warnings, repeated failures, and sudden spikes in log activity.


**Reasoning**:
I need to define the `AnomalyDetector` class, including its constructor to handle `anomaly_rules` and the `detect_anomalies` method to process `NormalizedLogEntry` objects, as specified in the instructions. This will involve iterating through logs and checking for patterns in log levels and messages.



In [ ]:
class AnomalyDetector:
    """Detects predefined abnormal events in a list of NormalizedLogEntry objects."""

    def __init__(self, anomaly_rules: dict = None):
        """
        Initializes the AnomalyDetector with a set of anomaly rules.

        Args:
            anomaly_rules (dict, optional): A dictionary where keys are log levels
                                            (e.g., 'ERROR', 'WARNING') or a special
                                            key like 'message_patterns' and values
                                            are lists of keywords or regex patterns to look for.
                                            Example: {'ERROR': ['failed', 'exception'],
                                                      'WARNING': ['timeout'],
                                                      'message_patterns': ['.*denial of service.*']}
        """
        self.anomaly_rules = anomaly_rules if anomaly_rules is not None else {
            'ERROR': ['error', 'fail', 'exception', 'critical', 'denied'],
            'WARNING': ['warn', 'timeout', 'unauthorized', 'suspicious'],
            'message_patterns': [] # For patterns that span across log levels or are more generic
        }
        print("AnomalyDetector initialized with rules.")

    def detect_anomalies(self, normalized_logs: list) -> list:
        """
        Detects anomalies in a list of NormalizedLogEntry objects based on configured rules.

        Args:
            normalized_logs (list): A list of NormalizedLogEntry objects.

        Returns:
            list: A list of NormalizedLogEntry objects identified as anomalies.
        """
        detected_anomalies = []
        for entry in normalized_logs:
            # Check by log level and associated keywords
            if entry.log_level.upper() in self.anomaly_rules:
                for keyword in self.anomaly_rules[entry.log_level.upper()]:
                    if keyword.lower() in entry.message.lower():
                        detected_anomalies.append(entry)
                        break # Move to next log entry once an anomaly is found

            # Check by generic message patterns
            if 'message_patterns' in self.anomaly_rules:
                for pattern in self.anomaly_rules['message_patterns']:
                    if re.search(pattern, entry.message, re.IGNORECASE):
                        if entry not in detected_anomalies: # Avoid duplicates if already added by log_level rule
                            detected_anomalies.append(entry)
                        break

        print(f"Detected {len(detected_anomalies)} anomalies out of {len(normalized_logs)} log entries.")
        return detected_anomalies

print("Defined AnomalyDetector class.")

Defined AnomalyDetector class.


## Design Incident Correlator

### Subtask:
Develop an `IncidentCorrelator` class that groups related `NormalizedLogEntry` objects into `Incident` objects. This should use configurable `time_windows` and other correlation rules (e.g., same host/service, similar error patterns) to link events into cohesive incidents.


**Reasoning**:
I need to define the `IncidentCorrelator` class with its constructor and the `correlate_incidents` method, adhering to the specified requirements for grouping `NormalizedLogEntry` objects into `Incident` objects based on time windows and correlation rules. This will involve sorting logs, iterating through them, and creating new `Incident` objects.



In [ ]:
import re
from datetime import datetime, timedelta

# Assuming NormalizedLogEntry and Incident classes are already defined
# from datetime import datetime
# class NormalizedLogEntry:
#     def __init__(self, timestamp: datetime, log_level: str, source: str, message: str):
#         self.timestamp = timestamp
#         self.log_level = log_level
#         self.source = source
#         self.message = message
#     def __repr__(self):
#         return f"NormalizedLogEntry(timestamp='{self.timestamp}', log_level='{self.log_level}', source='{self.source}', message='{self.message[:50]}...'")

# class Incident:
#     def __init__(
#         self,
#         start_time: datetime,
#         end_time: datetime,
#         affected_hosts_services: list,
#         total_events: int,
#         severity: str,
#         probable_root_cause: str
#     ):
#         self.start_time = start_time
#         self.end_time = end_time
#         self.affected_hosts_services = affected_hosts_services
#         self.total_events = total_events
#         self.severity = severity
#         self.probable_root_cause = probable_root_cause
#     def __repr__(self):
#         return (
#             f"Incident(start_time='{self.start_time}', end_time='{self.end_time}', "
#             f"affected_hosts_services={self.affected_hosts_services}, total_events={self.total_events}, "
#             f"severity='{self.severity}', probable_root_cause='{self.probable_root_cause[:50]}...')"
#         )


class IncidentCorrelator:
    """Groups related NormalizedLogEntry objects into Incident objects."""

    def __init__(
        self,
        time_window_seconds: int = 60,
        correlation_rules: dict = None
    ):
        """
        Initializes the IncidentCorrelator.

        Args:
            time_window_seconds (int): The time window in seconds within which
                                       log entries are considered related.
            correlation_rules (dict): A dictionary defining rules for correlation.
                                      Example: {'group_by_source': True, 'keyword_match': ['login failed']}
        """
        self.time_window = timedelta(seconds=time_window_seconds)
        self.correlation_rules = correlation_rules if correlation_rules is not None else {
            'group_by_source': True, # Group events from the same source/host
            'min_events_for_incident': 3, # Minimum events to form an incident
            'severity_keywords': { # Keywords that might indicate higher severity for probable root cause
                'CRITICAL': ['critical', 'failure', 'down'],
                'HIGH': ['error', 'exception', 'timeout'],
                'MEDIUM': ['warning', 'denied'],
                'LOW': ['info', 'debug']
            }
        }
        print(f"IncidentCorrelator initialized with time window: {self.time_window} and rules.")

    def _determine_severity_and_root_cause(self, correlated_entries: list) -> tuple:
        """
        Determines the overall severity and probable root cause for an incident.
        """
        if not correlated_entries: # Should not happen if called correctly
            return 'UNKNOWN', 'No correlated events.'

        # Initialize with lowest severity and a generic cause
        overall_severity = 'LOW'
        probable_root_cause = 'Multiple events detected.'

        # Prioritize severity based on log levels and then keywords
        severity_order = {'CRITICAL': 5, 'HIGH': 4, 'MEDIUM': 3, 'LOW': 2, 'UNKNOWN': 1}
        current_max_severity_score = 0

        # First pass: check log levels and find the highest severity
        for entry in correlated_entries:
            log_level_upper = entry.log_level.upper()
            if log_level_upper in severity_order and severity_order[log_level_upper] > current_max_severity_score:
                current_max_severity_score = severity_order[log_level_upper]
                overall_severity = log_level_upper
                probable_root_cause = f"High impact log level: {log_level_upper}. Message: {entry.message[:70]}..."

        # Second pass: refine severity and root cause based on message keywords within the determined severity range
        for entry in correlated_entries:
            for sev_level, keywords in self.correlation_rules.get('severity_keywords', {}).items():
                if severity_order.get(sev_level, 0) >= current_max_severity_score: # Only consider keywords for current or higher severity
                    for keyword in keywords:
                        if keyword.lower() in entry.message.lower():
                            # If a keyword points to a higher severity, update
                            if severity_order[sev_level] > current_max_severity_score:
                                current_max_severity_score = severity_order[sev_level]
                                overall_severity = sev_level
                                probable_root_cause = f"Keyword '{keyword}' found in message: {entry.message[:70]}..."

                            # If a keyword matches the current highest severity, refine root cause
                            elif severity_order[sev_level] == current_max_severity_score:
                                if len(probable_root_cause) < 100: # Try to get more specific cause if current is generic
                                    probable_root_cause = f"Keyword '{keyword}' found in message: {entry.message[:70]}..."

        # Fallback if no specific cause found
        if probable_root_cause == 'Multiple events detected.' and correlated_entries:
            # Take the message of the first high-severity event or just the first event
            for entry in correlated_entries:
                if entry.log_level.upper() in ['CRITICAL', 'HIGH', 'ERROR']:
                    probable_root_cause = f"First critical/error message: {entry.message[:70]}..."
                    break
            else:
                probable_root_cause = f"First message: {correlated_entries[0].message[:70]}..."

        return overall_severity, probable_root_cause


    def correlate_incidents(self, normalized_logs: list) -> list:
        """
        Groups related NormalizedLogEntry objects into Incident objects.

        Args:
            normalized_logs (list): A list of NormalizedLogEntry objects.

        Returns:
            list: A list of Incident objects.
        """
        if not normalized_logs:
            print("No logs to correlate.")
            return []

        # Sort logs by timestamp to process chronologically
        sorted_logs = sorted(normalized_logs, key=lambda x: x.timestamp)

        incidents = []
        processed_indices = set()

        for i, current_log in enumerate(sorted_logs):
            if i in processed_indices:
                continue

            # Start a new potential incident group
            current_incident_logs = [current_log]
            processed_indices.add(i)

            # Define the time window for this incident
            window_start_time = current_log.timestamp
            window_end_time = current_log.timestamp + self.time_window

            affected_hosts_services = {current_log.source}

            # Look for other logs within the time window and correlation rules
            for j in range(i + 1, len(sorted_logs)):
                next_log = sorted_logs[j]

                if j in processed_indices:
                    continue

                # Check if the log falls within the current incident's dynamic time window
                # The window expands with new logs, ensuring continuity
                if next_log.timestamp <= window_end_time:
                    # Apply correlation rules
                    is_related = True
                    if self.correlation_rules.get('group_by_source', True):
                        if next_log.source != current_log.source:
                            is_related = False

                    # Add more complex rules here, e.g., similar log levels, regex patterns
                    # For simplicity, we primarily group by source and time for now.
                    # A more advanced correlator would use pattern matching on messages.

                    if is_related:
                        current_incident_logs.append(next_log)
                        processed_indices.add(j)
                        affected_hosts_services.add(next_log.source)
                        # Extend the window if a new log pushes the end further
                        if next_log.timestamp > window_start_time + self.time_window:
                             window_end_time = next_log.timestamp + self.time_window

                # If the next log is too far in the future, it starts a new potential incident
                elif next_log.timestamp > window_end_time:
                    break # No more logs will fit this window

            # If enough logs are correlated, create an incident
            min_events = self.correlation_rules.get('min_events_for_incident', 1) # Default to 1 to allow all anomalies to be incidents
            if len(current_incident_logs) >= min_events:
                incident_start_time = min(log.timestamp for log in current_incident_logs)
                incident_end_time = max(log.timestamp for log in current_incident_logs)

                severity, probable_root_cause = self._determine_severity_and_root_cause(current_incident_logs)

                new_incident = Incident(
                    start_time=incident_start_time,
                    end_time=incident_end_time,
                    affected_hosts_services=list(affected_hosts_services),
                    total_events=len(current_incident_logs),
                    severity=severity,
                    probable_root_cause=probable_root_cause
                )
                incidents.append(new_incident)

        print(f"Correlated {len(normalized_logs)} logs into {len(incidents)} incidents.")
        return incidents

print("Defined IncidentCorrelator class.")

Defined IncidentCorrelator class.


## Create Report Generator

### Subtask:
Develop a `ReportGenerator` class to output the list of detected `Incident` objects into a structured incident report. This generator should support both CSV and JSON formats, summarizing all key incident details in a clear and human-readable manner.


**Reasoning**:
I need to define the `ReportGenerator` class as specified, including its constructor and methods for generating CSV and JSON reports. This involves handling `datetime` objects for JSON serialization and lists for CSV representation.



In [ ]:
import csv
import json
from datetime import datetime

# Assuming Incident class is already defined
# from datetime import datetime, timedelta
# class Incident:
#     def __init__(
#         self, start_time: datetime, end_time: datetime, affected_hosts_services: list,
#         total_events: int, severity: str, probable_root_cause: str
#     ):
#         self.start_time = start_time
#         self.end_time = end_time
#         self.affected_hosts_services = affected_hosts_services
#         self.total_events = total_events
#         self.severity = severity
#         self.probable_root_cause = probable_root_cause
#     def __repr__(self):
#         return (
#             f"Incident(start_time='{self.start_time}', end_time='{self.end_time}', "
#             f"affected_hosts_services={self.affected_hosts_services}, total_events={self.total_events}, "
#             f"severity='{self.severity}', probable_root_cause='{self.probable_root_cause[:50]}...')"
#         )


class ReportGenerator:
    """Generates structured incident reports in CSV and JSON formats."""

    def __init__(self):
        """Initializes the ReportGenerator."""
        print("ReportGenerator initialized.")

    def generate_csv_report(self, incidents: list, file_path: str):
        """
        Generates a CSV report from a list of Incident objects.

        Args:
            incidents (list): A list of Incident objects.
            file_path (str): The path to save the CSV file.
        """
        if not incidents:
            print("No incidents to report in CSV format.")
            return

        headers = [
            'start_time', 'end_time', 'affected_hosts_services',
            'total_events', 'severity', 'probable_root_cause'
        ]

        try:
            with open(file_path, 'w', newline='', encoding='utf-8') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(headers)
                for incident in incidents:
                    writer.writerow([
                        incident.start_time.isoformat(),
                        incident.end_time.isoformat(),
                        ', '.join(incident.affected_hosts_services), # Convert list to string
                        incident.total_events,
                        incident.severity,
                        incident.probable_root_cause
                    ])
            print(f"CSV report successfully generated and saved to: {file_path}")
        except IOError as e:
            print(f"Error writing CSV report to {file_path}: {e}")

    def generate_json_report(self, incidents: list, file_path: str):
        """
        Generates a JSON report from a list of Incident objects.

        Args:
            incidents (list): A list of Incident objects.
            file_path (str): The path to save the JSON file.
        """
        if not incidents:
            print("No incidents to report in JSON format.")
            return

        # Helper function to serialize datetime objects to ISO format
        def json_serial(obj):
            if isinstance(obj, datetime):
                return obj.isoformat()
            raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")

        incident_data = []
        for incident in incidents:
            incident_data.append({
                'start_time': incident.start_time,
                'end_time': incident.end_time,
                'affected_hosts_services': incident.affected_hosts_services,
                'total_events': incident.total_events,
                'severity': incident.severity,
                'probable_root_cause': incident.probable_root_cause
            })

        try:
            with open(file_path, 'w', encoding='utf-8') as jsonfile:
                json.dump(incident_data, jsonfile, indent=4, default=json_serial)
            print(f"JSON report successfully generated and saved to: {file_path}")
        except IOError as e:
            print(f"Error writing JSON report to {file_path}: {e}")

print("Defined ReportGenerator class.")

Defined ReportGenerator class.


## Implement Command-Line Interface (CLI)

### Subtask:
Set up an argparse-based command-line interface to allow users to specify input log files, desired output format (CSV/JSON), an optional output file path, and configurable parameters like the time window for correlation and the error threshold for incident triggering.


### Reasonings:
To implement the command-line interface as specified, I will create a code block that utilizes the `argparse` module. This will involve defining an `ArgumentParser` and adding various arguments for input file, output format, output path, correlation time window, minimum events for an incident, custom log format details, and an optional anomaly rules file. Finally, I will parse and print these arguments to ensure the setup is correct.

**Reasoning**:
I need to define a code block that imports `argparse`, sets up an argument parser with all specified arguments including types and defaults, parses the arguments, and then prints them to verify the setup for the CLI.



In [ ]:
import argparse
import json

def setup_cli_parser():
    """Sets up the argparse-based command-line interface."""
    parser = argparse.ArgumentParser(description='Log Analysis and Incident Correlation System')

    parser.add_argument(
        '--input_file',
        type=str,
        required=True,
        help='Path to the raw log file to be analyzed.'
    )
    parser.add_argument(
        '--output_format',
        type=str,
        choices=['csv', 'json'],
        default='csv',
        help='Desired report format (csv or json).'
    )
    parser.add_argument(
        '--output_path',
        type=str,
        default=None,
        help='Optional path to save the report file. If not provided, a default name will be used.'
    )
    parser.add_argument(
        '--time_window',
        type=int,
        default=60,
        help='Time window in seconds for incident correlation (e.g., 60 for 1 minute).'
    )
    parser.add_argument(
        '--min_events',
        type=int,
        default=1,
        help='Minimum number of events required to form an incident.'
    )
    parser.add_argument(
        '--log_format_regex',
        type=str,
        default=None,
        help='Custom log format regex pattern. Requires --log_format_mapping and --log_format_timestamp.'
    )
    parser.add_argument(
        '--log_format_mapping',
        type=str,
        default=None,
        help='JSON string representing a dictionary for custom log format mapping (e.g., {"group":"attribute"}).'
    )
    parser.add_argument(
        '--log_format_timestamp',
        type=str,
        default=None,
        help='Timestamp format string for custom log format (e.g., "%Y-%m-%d %H:%M:%S").'
    )
    parser.add_argument(
        '--anomaly_rules_file',
        type=str,
        default=None,
        help='Path to a JSON file containing custom anomaly detection rules.'
    )

    return parser

# Example of how to parse args, for testing the CLI setup
# In a real script, this would be part of the main execution block
if __name__ == '__main__':
    parser = setup_cli_parser()
    # For demonstration, we'll simulate parsing arguments
    # In actual use, this would be `args = parser.parse_args()`
    # For this notebook context, we'll just print a message about the parser.
    print("CLI ArgumentParser setup complete. Arguments can be parsed using `parser.parse_args()`.")
    print("Example usage: python your_script.py --input_file logs.txt --output_format json --time_window 120")


CLI ArgumentParser setup complete. Arguments can be parsed using `parser.parse_args()`.
Example usage: python your_script.py --input_file logs.txt --output_format json --time_window 120


**Reasoning**:
To complete the subtask, I need to demonstrate the parsing of command-line arguments and print them to verify the setup. Since this is a notebook environment, I will simulate arguments using `parse_args` with a list of strings and then print the resulting `args` object.



In [ ]:
import json

# Assuming setup_cli_parser function is defined in the previous cell
# If this were a standalone script, args = parser.parse_args()

# For demonstration in a notebook, we simulate command line arguments
parser = setup_cli_parser()

# Simulate arguments as if they were passed from the command line
simulated_args = [
    '--input_file', 'sample_logs.txt',
    '--output_format', 'json',
    '--output_path', 'report.json',
    '--time_window', '120',
    '--min_events', '5',
    '--log_format_regex', r'^(?P<timestamp>\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}) (?P<log_level>\w+) (?P<source>\w+): (?P<message>.*)$',
    '--log_format_mapping', '{"timestamp":"timestamp", "log_level":"log_level", "source":"source", "message":"message"}',
    '--log_format_timestamp', '%Y-%m-%d %H:%M:%S',
    '--anomaly_rules_file', 'custom_rules.json'
]

# Parse the simulated arguments
args = parser.parse_args(simulated_args)

print("Parsed Arguments:")
for arg_name, arg_value in vars(args).items():
    # Attempt to parse JSON strings for log_format_mapping if present
    if arg_name == 'log_format_mapping' and arg_value is not None:
        try:
            parsed_value = json.loads(arg_value)
            print(f"  {arg_name}: {parsed_value} (type: {type(parsed_value)})")
        except json.JSONDecodeError:
            print(f"  {arg_name}: {arg_value} (type: {type(arg_value)}) - Invalid JSON")
    else:
        print(f"  {arg_name}: {arg_value} (type: {type(arg_value)})")

print("CLI argument parsing verification complete.")

Parsed Arguments:
  input_file: sample_logs.txt (type: <class 'str'>)
  output_format: json (type: <class 'str'>)
  output_path: report.json (type: <class 'str'>)
  time_window: 120 (type: <class 'int'>)
  min_events: 5 (type: <class 'int'>)
  log_format_regex: ^(?P<timestamp>\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}) (?P<log_level>\w+) (?P<source>\w+): (?P<message>.*)$ (type: <class 'str'>)
  log_format_mapping: {'timestamp': 'timestamp', 'log_level': 'log_level', 'source': 'source', 'message': 'message'} (type: <class 'dict'>)
  log_format_timestamp: %Y-%m-%d %H:%M:%S (type: <class 'str'>)
  anomaly_rules_file: custom_rules.json (type: <class 'str'>)
CLI argument parsing verification complete.


## Implement Visualization Functionality

### Subtask:
Implement the bonus functionality to generate basic visualizations. This includes plotting incident timelines to show when incidents occurred and their duration, and visualizing error trends over time to identify patterns or spikes, possibly using `matplotlib`.


**Reasoning**:
I need to define the `VisualizationGenerator` class, including its constructor and methods for plotting incident timelines and error trends, as specified in the instructions. This will involve using `matplotlib.pyplot` and handling `datetime` objects for plotting.



In [ ]:
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Assuming NormalizedLogEntry and Incident classes are already defined
# from datetime import datetime
# class NormalizedLogEntry:
#     def __init__(self, timestamp: datetime, log_level: str, source: str, message: str):
#         self.timestamp = timestamp
#         self.log_level = log_level
#         self.source = source
#         self.message = message

# class Incident:
#     def __init__(
#         self, start_time: datetime, end_time: datetime, affected_hosts_services: list,
#         total_events: int, severity: str, probable_root_cause: str
#     ):
#         self.start_time = start_time
#         self.end_time = end_time
#         self.affected_hosts_services = affected_hosts_services
#         self.total_events = total_events
#         self.severity = severity
#         self.probable_root_cause = probable_root_cause


class VisualizationGenerator:
    """Generates visualizations for incident timelines and error trends."""

    def __init__(self):
        """Initializes the VisualizationGenerator."""
        print("VisualizationGenerator initialized.")

    def plot_incident_timeline(self, incidents: list, file_path: str = 'incident_timeline.png'):
        """
        Generates a timeline plot of incidents.

        Args:
            incidents (list): A list of Incident objects.
            file_path (str): The path to save the plot file.
        """
        if not incidents:
            print("No incidents to plot for timeline.")
            return

        # Sort incidents by start time for better visualization
        incidents.sort(key=lambda x: x.start_time)

        fig, ax = plt.subplots(figsize=(12, 6))

        # Define colors for severity levels
        severity_colors = {
            'CRITICAL': 'darkred',
            'HIGH': 'red',
            'MEDIUM': 'orange',
            'LOW': 'green',
            'UNKNOWN': 'gray'
        }

        # Prepare data for plotting
        y_positions = range(len(incidents))
        bars = []
        labels = []
        for i, incident in enumerate(incidents):
            duration = (incident.end_time - incident.start_time).total_seconds() / 3600 # duration in hours
            # If duration is 0 (start_time == end_time), give it a small positive value for visibility
            if duration == 0:
                duration = 0.01 # represent as 36 seconds minimum
            color = severity_colors.get(incident.severity.upper(), 'gray')
            ax.barbs(incident.start_time, y_positions[i], 0, duration, color=color, length=0.8, fill_empty=True, pivot='middle', label=incident.severity.upper() if incident.severity.upper() not in labels else "")
            bars.append(ax.barh(y_positions[i], width=duration, left=incident.start_time, height=0.6, color=color, label=incident.severity.upper() if incident.severity.upper() not in labels else ""))
            labels.append(incident.severity.upper())

        ax.set_yticks(y_positions)
        ax.set_yticklabels([f"Incident {i+1}" for i in range(len(incidents))])

        # Format x-axis as datetime
        fig.autofmt_xdate()
        ax.set_xlabel('Time')
        ax.set_ylabel('Incident')
        ax.set_title('Incident Timeline by Duration and Severity')

        # Create custom legend for severity colors
        unique_severities = sorted(list(set([inc.severity.upper() for inc in incidents])), key=lambda x: severity_colors.get(x, 'gray'))
        legend_handles = [plt.Rectangle((0,0),1,1, color=severity_colors.get(s, 'gray')) for s in unique_severities]
        ax.legend(legend_handles, unique_severities, title='Severity', loc='upper left', bbox_to_anchor=(1, 1))

        plt.tight_layout()
        try:
            plt.savefig(file_path)
            print(f"Incident timeline plot saved to: {file_path}")
        except Exception as e:
            print(f"Error saving incident timeline plot to {file_path}: {e}")
        plt.close()

    def plot_error_trends(self, normalized_logs: list, interval_minutes: int = 60, file_path: str = 'error_trends.png'):
        """
        Generates a plot visualizing error/warning trends over time.

        Args:
            normalized_logs (list): A list of NormalizedLogEntry objects.
            interval_minutes (int): Time interval in minutes for grouping logs.
            file_path (str): The path to save the plot file.
        """
        if not normalized_logs:
            print("No logs to plot for error trends.")
            return

        # Filter for error/warning/critical logs
        error_logs = [log for log in normalized_logs if log.log_level.upper() in ['ERROR', 'WARNING', 'CRITICAL']]
        if not error_logs:
            print("No error, warning, or critical logs to plot for trends.")
            return

        # Sort logs by timestamp
        error_logs.sort(key=lambda x: x.timestamp)

        # Determine time bins
        start_time = min(log.timestamp for log in error_logs)
        end_time = max(log.timestamp for log in error_logs)
        time_delta = timedelta(minutes=interval_minutes)

        current_bin_start = start_time
        time_bins = []
        while current_bin_start <= end_time + time_delta: # Ensure the last bin is included
            time_bins.append(current_bin_start)
            current_bin_start += time_delta

        # Count logs per bin and per log level
        trend_data = {level: [0] * (len(time_bins) - 1) for level in ['CRITICAL', 'ERROR', 'WARNING']}
        bin_labels = [tb.strftime('%Y-%m-%d %H:%M') for tb in time_bins[:-1]]

        for log in error_logs:
            for i in range(len(time_bins) - 1):
                if time_bins[i] <= log.timestamp < time_bins[i+1]:
                    log_level_upper = log.log_level.upper()
                    if log_level_upper in trend_data:
                        trend_data[log_level_upper][i] += 1
                    break

        fig, ax = plt.subplots(figsize=(14, 7))

        # Plot each log level trend
        colors = {'CRITICAL': 'darkred', 'ERROR': 'red', 'WARNING': 'orange'}
        for level in ['CRITICAL', 'ERROR', 'WARNING']:
            if any(trend_data[level]): # Only plot if there's data for this level
                ax.plot(bin_labels, trend_data[level], label=level, color=colors.get(level))

        ax.set_xlabel('Time')
        ax.set_ylabel(f'Number of Events per {interval_minutes} minutes')
        ax.set_title('Error and Warning Trends Over Time')
        ax.legend()
        fig.autofmt_xdate()
        plt.grid(True)
        plt.tight_layout()

        try:
            plt.savefig(file_path)
            print(f"Error trends plot saved to: {file_path}")
        except Exception as e:
            print(f"Error saving error trends plot to {file_path}: {e}")
        plt.close()

print("Defined VisualizationGenerator class.")

Defined VisualizationGenerator class.


## Final Task

### Subtask:
Provide a summary of the implemented log analysis and incident correlation system, its capabilities, instructions on how to use it via the command-line interface, and an overview of the generated reports and visualizations.


## Summary:

### Data Analysis Key Findings

*   **Data Structures Defined**: Core data structures, `NormalizedLogEntry` and `Incident`, were successfully defined to represent parsed log data and correlated incidents, respectively.
*   **Flexible Log Parsing**: A `LogParser` class was implemented, capable of parsing diverse log formats using regular expressions, extracting essential fields like timestamp, log level, source, and message. It supports adding custom log formats and includes robust timestamp conversion.
*   **Centralized Log Normalization**: The `LogNormalizer` class was developed to standardize raw log lines from various sources into the `NormalizedLogEntry` format, ensuring consistency for subsequent analysis.
*   **Configurable Anomaly Detection**: An `AnomalyDetector` class was created to identify abnormal events based on predefined rules, including log levels, keywords, and message patterns, allowing for flexible anomaly identification.
*   **Sophisticated Incident Correlation**: The `IncidentCorrelator` class was designed to group related `NormalizedLogEntry` objects into `Incident` objects. It uses configurable time windows, applies correlation rules (e.g., grouping by source), and intelligently determines incident severity and probable root cause.
*   **Structured Report Generation**: A `ReportGenerator` class was implemented to produce structured incident reports in both CSV and JSON formats, summarizing key incident details in a human-readable manner and handling data serialization for different output types.
*   **User-Friendly Command-Line Interface**: An `argparse`-based Command-Line Interface (CLI) was set up, enabling users to easily configure input files, output formats, correlation parameters (like time window and minimum events), custom log formats, and anomaly detection rules.
*   **Visual Incident and Trend Analysis**: A `VisualizationGenerator` class was developed to create visual reports using `matplotlib`. It can plot incident timelines, showing duration and severity, and visualize error and warning trends over time to highlight patterns and spikes.

### Insights or Next Steps

*   The developed system provides a robust and modular framework for end-to-end log analysis and incident correlation, offering significant value in proactively identifying and addressing system issues.
*   Future enhancements could include implementing machine learning techniques for anomaly detection to identify more complex, non-predefined patterns, and integrating with alerting systems for real-time incident notification.


# Task
The user has approved the previous steps. Now, let's proceed with setting up the web framework.

## Setup Web Framework

### Subtask:
Choose and set up a basic Python web framework (e.g., Flask or FastAPI) to serve as the backend for the application. This step will involve installing the framework and creating the initial application structure. We will use `Flask` for its simplicity and ease of setup.

### Reasoning:
To set up the web framework, I will first install Flask. Then, I will create a basic Flask application structure, including an `app.py` file to initialize the Flask app and a simple route to verify it's working. This will lay the foundation for integrating the log analysis components.

## Setup Web Framework

### Subtask:
Choose and set up a basic Python web framework (e.g., Flask or FastAPI) to serve as the backend for the application. This step will involve installing the framework and creating the initial application structure.


**Reasoning**:
First, I need to install the Flask library as instructed, which is a prerequisite for setting up the web framework.



**Reasoning**:
The previous attempt to install Flask resulted in a `SyntaxError` because `pip install` is a shell command and needs to be prefixed with `!` when executed within a Python code cell in an IPython/Jupyter environment. I will correct this to ensure proper installation.



# Task
To set up a Flask web framework, install the Flask library. This step will involve correcting the previous `pip install Flask` command to `!pip install Flask` to properly execute it in the Colab environment.

## Setup Web Framework

### Subtask:
Install the Flask library to set up a basic Python web framework.


**Reasoning**:
The previous attempt to install Flask failed because `pip install` was not prefixed with `!`. I will correct this to properly install the Flask library.



In [2]:
get_ipython().system('pip install Flask')
print("Flask library installed.")

Flask library installed.


**Reasoning**:
With Flask installed, I will now create a basic Flask application in this notebook cell, defining a simple route to verify its functionality. This will serve as the initial application structure for the web framework.



In [3]:
from flask import Flask

app = Flask(__name__)

@app.route('/')
def hello_world():
    return 'Hello, World! Flask is running.'

print("Flask application 'app' created with a '/' route.")
print("To run this Flask app from a Python file (e.g., app.py), you would typically use:")
print("export FLASK_APP=app.py")
print("flask run")
print("In a Colab environment, running Flask requires additional steps for public access, which will be covered later if needed.")

Flask application 'app' created with a '/' route.
To run this Flask app from a Python file (e.g., app.py), you would typically use:
export FLASK_APP=app.py
flask run
In a Colab environment, running Flask requires additional steps for public access, which will be covered later if needed.


## Define API Endpoints

### Subtask:
Create API endpoints within the chosen web framework to handle different functionalities. This includes endpoints for uploading log files, triggering the log analysis pipeline, and retrieving generated reports (CSV/JSON) and visualizations.


**Reasoning**:
I need to define the new API endpoints within the Flask application. This involves adding routes for file uploads, triggering analysis, and retrieving reports and visualizations, importing necessary Flask components like `request`, `jsonify`, and `send_file`.



In [4]:
from flask import Flask, request, jsonify, send_file
import os

# Assuming 'app' is already defined from a previous cell
# app = Flask(__name__)

# Create a directory for uploads and reports if it doesn't exist
UPLOAD_FOLDER = 'uploads'
REPORTS_FOLDER = 'reports'
VISUALIZATIONS_FOLDER = 'visualizations'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(REPORTS_FOLDER, exist_ok=True)
os.makedirs(VISUALIZATIONS_FOLDER, exist_ok=True)

@app.route('/upload', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        return jsonify({'error': 'No file part in the request'}), 400
    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400
    if file:
        filename = os.path.join(UPLOAD_FOLDER, file.filename)
        file.save(filename)
        return jsonify({'message': f'File {file.filename} uploaded successfully', 'filepath': filename}), 200
    return jsonify({'error': 'File upload failed'}), 500

@app.route('/analyze', methods=['POST'])
def analyze_logs():
    # This endpoint will trigger the full log analysis pipeline
    # It should receive parameters for analysis (e.g., input_file_path, correlation_time_window, etc.)
    # For now, it's a placeholder. The actual logic will involve calling the LogParser, LogNormalizer, etc.
    data = request.get_json()
    if not data or 'input_file' not in data:
        return jsonify({'error': 'Missing input_file in request body'}), 400

    input_file_path = data['input_file']
    # In a real scenario, validate input_file_path and ensure it's in UPLOAD_FOLDER

    # Placeholder for running the full analysis pipeline
    # result = run_analysis_pipeline(input_file_path, ...)

    return jsonify({'message': f'Analysis triggered for {input_file_path}', 'status': 'processing'}), 202

@app.route('/reports/<format>', methods=['GET'])
def get_report(format):
    if format not in ['csv', 'json']:
        return jsonify({'error': 'Invalid report format. Choose csv or json.'}), 400

    # Placeholder for actual report generation/retrieval logic
    # In a real scenario, reports would be generated after analysis and stored.
    report_filename = f'incident_report.{format}'
    report_path = os.path.join(REPORTS_FOLDER, report_filename)

    if not os.path.exists(report_path):
        return jsonify({'error': f'Report not found for format {format}. Please run analysis first.'}), 404

    return send_file(report_path, as_attachment=True, download_name=report_filename)

@app.route('/visualizations/<type>', methods=['GET'])
def get_visualization(type):
    if type not in ['timeline', 'trends']:
        return jsonify({'error': 'Invalid visualization type. Choose timeline or trends.'}), 400

    # Placeholder for actual visualization generation/retrieval logic
    # Visualizations would be generated after analysis and stored as image files.
    viz_filename = f'{type}_visualization.png'
    viz_path = os.path.join(VISUALIZATIONS_FOLDER, viz_filename)

    if not os.path.exists(viz_path):
        return jsonify({'error': f'Visualization not found for type {type}. Please run analysis first.'}), 404

    return send_file(viz_path, mimetype='image/png')

print("API endpoints for upload, analyze, reports, and visualizations defined.")

API endpoints for upload, analyze, reports, and visualizations defined.


## Integrate Log Analysis Components

### Subtask:
Adapt and integrate the existing `LogParser`, `LogNormalizer`, `AnomalyDetector`, `IncidentCorrelator`, `ReportGenerator`, and `VisualizationGenerator` classes into the web application's logic. These components will be called by the API endpoints to perform the analysis.


**Reasoning**:
I need to integrate all log analysis components into the Flask application. This involves importing the necessary classes, defining a comprehensive `run_analysis_pipeline` function that orchestrates the entire workflow, and then modifying the `/analyze` API endpoint to call this pipeline and return the generated report and visualization paths.



**Reasoning**:
The previous code failed because the `/analyze` endpoint was being redefined using `@app.route` in a subsequent cell, leading to an `AssertionError`. To fix this, I will define the new logic for the `analyze_logs` function and then explicitly assign it to `app.view_functions['analyze_logs']`, replacing the old function without re-registering the route.



In [6]:
from flask import Flask, request, jsonify, send_file
import os
import json
from datetime import datetime, timedelta
import re
import csv
import matplotlib.pyplot as plt

# --- Re-defining classes from previous steps for clarity and self-containment in a single cell execution ---
# (In a real application, these would be imported from separate modules)

class NormalizedLogEntry:
    """Represents a normalized log entry."""
    def __init__(self, timestamp: datetime, log_level: str, source: str, message: str):
        self.timestamp = timestamp
        self.log_level = log_level
        self.source = source
        self.message = message

    def __repr__(self):
        return f"NormalizedLogEntry(timestamp='{self.timestamp}', log_level='{self.log_level}', source='{self.source}', message='{self.message[:50]}...')"

class Incident:
    """Represents a correlated incident."""
    def __init__(
        self,
        start_time: datetime,
        end_time: datetime,
        affected_hosts_services: list,
        total_events: int,
        severity: str,
        probable_root_cause: str
    ):
        self.start_time = start_time
        self.end_time = end_time
        self.affected_hosts_services = affected_hosts_services
        self.total_events = total_events
        self.severity = severity
        self.probable_root_cause = probable_root_cause

    def __repr__(self):
        return (
            f"Incident(start_time='{self.start_time}', end_time='{self.end_time}', "
            f"affected_hosts_services={self.affected_hosts_services}, total_events={self.total_events}, "
            f"severity='{self.severity}', probable_root_cause='{self.probable_root_cause[:50]}...')"
        )


class LogParser:
    """Parses raw log lines into NormalizedLogEntry objects based on configured formats."""

    def __init__(self, log_formats: list = None):
        self.log_formats = log_formats if log_formats is not None else []

    def add_format(self, regex_pattern: str, mapping: dict, timestamp_format: str = None):
        self.log_formats.append({
            'regex': re.compile(regex_pattern),
            'mapping': mapping,
            'timestamp_format': timestamp_format
        })

    def parse_log_entry(self, log_line: str) -> NormalizedLogEntry or None:
        for fmt in self.log_formats:
            match = fmt['regex'].match(log_line)
            if match:
                extracted_data = match.groupdict()
                parsed_attrs = {}
                for group_name, attr_name in fmt['mapping'].items():
                    if group_name in extracted_data:
                        parsed_attrs[attr_name] = extracted_data[group_name]
                timestamp_str = parsed_attrs.get('timestamp')
                if timestamp_str:
                    try:
                        if fmt['timestamp_format']:
                            parsed_attrs['timestamp'] = datetime.strptime(timestamp_str, fmt['timestamp_format'])
                        else:
                            try:
                                parsed_attrs['timestamp'] = datetime.fromisoformat(timestamp_str)
                            except ValueError:
                                if 'T' in timestamp_str and '+' in timestamp_str:
                                    parts = timestamp_str.split('+')
                                    timestamp_no_tz = parts[0]
                                    if '.' in timestamp_no_tz:
                                        pre_dot, post_dot = timestamp_no_tz.split('.')
                                        if len(post_dot) > 6:
                                            timestamp_no_tz = f"{pre_dot}.{post_dot[:6]}"
                                    parsed_attrs['timestamp'] = datetime.strptime(timestamp_no_tz, '%Y-%m-%dT%H:%M:%S.%f')
                                else:
                                    parsed_attrs['timestamp'] = datetime.strptime(timestamp_str, '%Y-%m-%d %H:%M:%S')
                    except ValueError as e:
                        print(f"Warning: Could not parse timestamp '{timestamp_str}' with format '{fmt.get('timestamp_format', 'auto')}': {e}")
                        return None
                else:
                    print("Warning: Timestamp not found in parsed log entry.")
                    return None

                timestamp = parsed_attrs.get('timestamp')
                log_level = parsed_attrs.get('log_level', 'UNKNOWN')
                source = parsed_attrs.get('source', 'UNKNOWN')
                message = parsed_attrs.get('message', log_line)

                if timestamp is None:
                    return None

                return NormalizedLogEntry(timestamp, log_level, source, message)
        return None


class LogNormalizer:
    """Normalizes raw log lines into a list of NormalizedLogEntry objects using a LogParser."""

    def __init__(self):
        pass

    def normalize_logs(self, log_parser: LogParser, raw_log_lines: list) -> list:
        normalized_entries = []
        for line in raw_log_lines:
            entry = log_parser.parse_log_entry(line)
            if entry:
                normalized_entries.append(entry)
        return normalized_entries


class AnomalyDetector:
    """Detects predefined abnormal events in a list of NormalizedLogEntry objects."""

    def __init__(self, anomaly_rules: dict = None):
        self.anomaly_rules = anomaly_rules if anomaly_rules is not None else {
            'ERROR': ['error', 'fail', 'exception', 'critical', 'denied'],
            'WARNING': ['warn', 'timeout', 'unauthorized', 'suspicious'],
            'message_patterns': []
        }

    def detect_anomalies(self, normalized_logs: list) -> list:
        detected_anomalies = []
        for entry in normalized_logs:
            if entry.log_level.upper() in self.anomaly_rules:
                for keyword in self.anomaly_rules[entry.log_level.upper()]:
                    if keyword.lower() in entry.message.lower():
                        detected_anomalies.append(entry)
                        break
            if 'message_patterns' in self.anomaly_rules:
                for pattern in self.anomaly_rules['message_patterns']:
                    if re.search(pattern, entry.message, re.IGNORECASE):
                        if entry not in detected_anomalies:
                            detected_anomalies.append(entry)
                        break
        return detected_anomalies


class IncidentCorrelator:
    """Groups related NormalizedLogEntry objects into Incident objects."""

    def __init__(
        self,
        time_window_seconds: int = 60,
        correlation_rules: dict = None
    ):
        self.time_window = timedelta(seconds=time_window_seconds)
        self.correlation_rules = correlation_rules if correlation_rules is not None else {
            'group_by_source': True,
            'min_events_for_incident': 3,
            'severity_keywords': {
                'CRITICAL': ['critical', 'failure', 'down'],
                'HIGH': ['error', 'exception', 'timeout'],
                'MEDIUM': ['warning', 'denied'],
                'LOW': ['info', 'debug']
            }
        }

    def _determine_severity_and_root_cause(self, correlated_entries: list) -> tuple:
        if not correlated_entries:
            return 'UNKNOWN', 'No correlated events.'
        overall_severity = 'LOW'
        probable_root_cause = 'Multiple events detected.'
        severity_order = {'CRITICAL': 5, 'HIGH': 4, 'MEDIUM': 3, 'LOW': 2, 'UNKNOWN': 1}
        current_max_severity_score = 0

        for entry in correlated_entries:
            log_level_upper = entry.log_level.upper()
            if log_level_upper in severity_order and severity_order[log_level_upper] > current_max_severity_score:
                current_max_severity_score = severity_order[log_level_upper]
                overall_severity = log_level_upper
                probable_root_cause = f"High impact log level: {log_level_upper}. Message: {entry.message[:70]}..."

        for entry in correlated_entries:
            for sev_level, keywords in self.correlation_rules.get('severity_keywords', {}).items():
                if severity_order.get(sev_level, 0) >= current_max_severity_score:
                    for keyword in keywords:
                        if keyword.lower() in entry.message.lower():
                            if severity_order[sev_level] > current_max_severity_score:
                                current_max_severity_score = severity_order[sev_level]
                                overall_severity = sev_level
                                probable_root_cause = f"Keyword '{keyword}' found in message: {entry.message[:70]}..."
                            elif severity_order[sev_level] == current_max_severity_score:
                                if len(probable_root_cause) < 100: # Try to get more specific cause if current is generic
                                    probable_root_cause = f"Keyword '{keyword}' found in message: {entry.message[:70]}..."

        if probable_root_cause == 'Multiple events detected.' and correlated_entries:
            for entry in correlated_entries:
                if entry.log_level.upper() in ['CRITICAL', 'HIGH', 'ERROR']:
                    probable_root_cause = f"First critical/error message: {entry.message[:70]}..."
                    break
            else:
                probable_root_cause = f"First message: {correlated_entries[0].message[:70]}..."

        return overall_severity, probable_root_cause


    def correlate_incidents(self, normalized_logs: list) -> list:
        if not normalized_logs:
            return []
        sorted_logs = sorted(normalized_logs, key=lambda x: x.timestamp)
        incidents = []
        processed_indices = set()

        for i, current_log in enumerate(sorted_logs):
            if i in processed_indices:
                continue

            current_incident_logs = [current_log]
            processed_indices.add(i)

            window_start_time = current_log.timestamp
            window_end_time = current_log.timestamp + self.time_window

            affected_hosts_services = {current_log.source}

            for j in range(i + 1, len(sorted_logs)):
                next_log = sorted_logs[j]

                if j in processed_indices:
                    continue

                if next_log.timestamp <= window_end_time:
                    is_related = True
                    if self.correlation_rules.get('group_by_source', True):
                        if next_log.source != current_log.source:
                            is_related = False

                    if is_related:
                        current_incident_logs.append(next_log)
                        processed_indices.add(j)
                        affected_hosts_services.add(next_log.source)
                        if next_log.timestamp > window_start_time + self.time_window:
                             window_end_time = next_log.timestamp + self.time_window
                elif next_log.timestamp > window_end_time:
                    break

            min_events = self.correlation_rules.get('min_events_for_incident', 1)
            if len(current_incident_logs) >= min_events:
                incident_start_time = min(log.timestamp for log in current_incident_logs)
                incident_end_time = max(log.timestamp for log in current_incident_logs)

                severity, probable_root_cause = self._determine_severity_and_root_cause(current_incident_logs)

                new_incident = Incident(
                    start_time=incident_start_time,
                    end_time=incident_end_time,
                    affected_hosts_services=list(affected_hosts_services),
                    total_events=len(current_incident_logs),
                    severity=severity,
                    probable_root_cause=probable_root_cause
                )
                incidents.append(new_incident)

        return incidents


class ReportGenerator:
    """Generates structured incident reports in CSV and JSON formats."""

    def __init__(self):
        pass

    def generate_csv_report(self, incidents: list, file_path: str):
        if not incidents:
            return
        headers = [
            'start_time', 'end_time', 'affected_hosts_services',
            'total_events', 'severity', 'probable_root_cause'
        ]
        try:
            with open(file_path, 'w', newline='', encoding='utf-8') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(headers)
                for incident in incidents:
                    writer.writerow([
                        incident.start_time.isoformat(),
                        incident.end_time.isoformat(),
                        ', '.join(incident.affected_hosts_services),
                        incident.total_events,
                        incident.severity,
                        incident.probable_root_cause
                    ])
        except IOError as e:
            print(f"Error writing CSV report to {file_path}: {e}")

    def generate_json_report(self, incidents: list, file_path: str):
        if not incidents:
            return
        def json_serial(obj):
            if isinstance(obj, datetime):
                return obj.isoformat()
            raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")

        incident_data = []
        for incident in incidents:
            incident_data.append({
                'start_time': incident.start_time,
                'end_time': incident.end_time,
                'affected_hosts_services': incident.affected_hosts_services,
                'total_events': incident.total_events,
                'severity': incident.severity,
                'probable_root_cause': incident.probable_root_cause
            })
        try:
            with open(file_path, 'w', encoding='utf-8') as jsonfile:
                json.dump(incident_data, jsonfile, indent=4, default=json_serial)
        except IOError as e:
            print(f"Error writing JSON report to {file_path}: {e}")


class VisualizationGenerator:
    """Generates visualizations for incident timelines and error trends."""

    def __init__(self):
        pass

    def plot_incident_timeline(self, incidents: list, file_path: str = 'incident_timeline.png'):
        if not incidents:
            return
        incidents.sort(key=lambda x: x.start_time)
        fig, ax = plt.subplots(figsize=(12, 6))
        severity_colors = {
            'CRITICAL': 'darkred', 'HIGH': 'red', 'MEDIUM': 'orange',
            'LOW': 'green', 'UNKNOWN': 'gray'
        }
        y_positions = range(len(incidents))
        for i, incident in enumerate(incidents):
            duration = (incident.end_time - incident.start_time).total_seconds() / 3600
            if duration == 0:
                duration = 0.01
            color = severity_colors.get(incident.severity.upper(), 'gray')
            ax.barh(y_positions[i], width=duration, left=incident.start_time, height=0.6, color=color)
        ax.set_yticks(y_positions)
        ax.set_yticklabels([f"Incident {i+1}" for i in range(len(incidents))])
        fig.autofmt_xdate()
        ax.set_xlabel('Time')
        ax.set_ylabel('Incident')
        ax.set_title('Incident Timeline by Duration and Severity')
        unique_severities = sorted(list(set([inc.severity.upper() for inc in incidents])), key=lambda x: severity_colors.get(x, 'gray'))
        legend_handles = [plt.Rectangle((0,0),1,1, color=severity_colors.get(s, 'gray')) for s in unique_severities]
        ax.legend(legend_handles, unique_severities, title='Severity', loc='upper left', bbox_to_anchor=(1, 1))
        plt.tight_layout()
        try:
            plt.savefig(file_path)
        except Exception as e:
            print(f"Error saving incident timeline plot to {file_path}: {e}")
        plt.close()

    def plot_error_trends(self, normalized_logs: list, interval_minutes: int = 60, file_path: str = 'error_trends.png'):
        if not normalized_logs:
            return
        error_logs = [log for log in normalized_logs if log.log_level.upper() in ['ERROR', 'WARNING', 'CRITICAL']]
        if not error_logs:
            return
        error_logs.sort(key=lambda x: x.timestamp)
        start_time = min(log.timestamp for log in error_logs)
        end_time = max(log.timestamp for log in error_logs)
        time_delta = timedelta(minutes=interval_minutes)
        current_bin_start = start_time
        time_bins = []
        while current_bin_start <= end_time + time_delta:
            time_bins.append(current_bin_start)
            current_bin_start += time_delta
        trend_data = {level: [0] * (len(time_bins) - 1) for level in ['CRITICAL', 'ERROR', 'WARNING']}
        bin_labels = [tb.strftime('%Y-%m-%d %H:%M') for tb in time_bins[:-1]]

        for log in error_logs:
            for i in range(len(time_bins) - 1):
                if time_bins[i] <= log.timestamp < time_bins[i+1]:
                    log_level_upper = log.log_level.upper()
                    if log_level_upper in trend_data:
                        trend_data[log_level_upper][i] += 1
                    break

        fig, ax = plt.subplots(figsize=(14, 7))
        colors = {'CRITICAL': 'darkred', 'ERROR': 'red', 'WARNING': 'orange'}
        for level in ['CRITICAL', 'ERROR', 'WARNING']:
            if any(trend_data[level]):
                ax.plot(bin_labels, trend_data[level], label=level, color=colors.get(level))
        ax.set_xlabel('Time')
        ax.set_ylabel(f'Number of Events per {interval_minutes} minutes')
        ax.set_title('Error and Warning Trends Over Time')
        ax.legend()
        fig.autofmt_xdate()
        plt.grid(True)
        plt.tight_layout()
        try:
            plt.savefig(file_path)
        except Exception as e:
            print(f"Error saving error trends plot to {file_path}: {e}")
        plt.close()


# Assuming 'app' is already defined from a previous cell
# app = Flask(__name__)

# Create a directory for uploads and reports if it doesn't exist
# UPLOAD_FOLDER, REPORTS_FOLDER, VISUALIZATIONS_FOLDER are assumed to be defined globally or in the previous cell
# For robustness, we re-define them here if they might not be in scope for a fresh run
UPLOAD_FOLDER = os.environ.get('UPLOAD_FOLDER', 'uploads')
REPORTS_FOLDER = os.environ.get('REPORTS_FOLDER', 'reports')
VISUALIZATIONS_FOLDER = os.environ.get('VISUALIZATIONS_FOLDER', 'visualizations')
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(REPORTS_FOLDER, exist_ok=True)
os.makedirs(VISUALIZATIONS_FOLDER, exist_ok=True)

def run_analysis_pipeline(
    input_file_path: str,
    time_window: int = 60,
    min_events: int = 1,
    log_format_regex: str = None,
    log_format_mapping: str = None, # JSON string
    log_format_timestamp: str = None,
    anomaly_rules_file: str = None
) -> dict:
    """Encapsulates the entire log analysis workflow."""
    print(f"Starting analysis for: {input_file_path}")
    results = {
        'csv_report': None,
        'json_report': None,
        'incident_timeline_plot': None,
        'error_trends_plot': None,
        'message': 'Analysis completed successfully.'
    }

    # 1. Read raw log lines
    try:
        with open(input_file_path, 'r', encoding='utf-8') as f:
            raw_log_lines = f.readlines()
    except FileNotFoundError:
        results['message'] = f"Error: Input file not found at {input_file_path}"
        return results
    except Exception as e:
        results['message'] = f"Error reading input file: {e}"
        return results

    # 2. Initialize LogParser and add custom format if provided
    log_parser = LogParser()
    # Add a default generic log format if no custom format is provided and no other formats exist
    # For simplicity, we assume one of these will always match or a custom one is provided
    # A more robust solution might try multiple default formats.
    default_regex = r'^(?P<timestamp>\S+ \S+ \d+ \d{2}:\d{2}:\d{2}) (?P<source>\S+) (?P<log_level>\S+): (?P<message>.*)$'
    default_mapping = {'timestamp': 'timestamp', 'source': 'source', 'log_level': 'log_level', 'message': 'message'}
    default_timestamp_format = '%b %d %H:%M:%S'
    log_parser.add_format(default_regex, default_mapping, default_timestamp_format)

    if log_format_regex and log_format_mapping:
        try:
            parsed_mapping = json.loads(log_format_mapping)
            log_parser.add_format(log_format_regex, parsed_mapping, log_format_timestamp)
        except json.JSONDecodeError:
            results['message'] = "Error: Invalid JSON for log_format_mapping."
            return results
        except Exception as e:
            results['message'] = f"Error adding custom log format: {e}"
            return results

    # 3. Normalize logs
    log_normalizer = LogNormalizer()
    normalized_entries = log_normalizer.normalize_logs(log_parser, raw_log_lines)
    if not normalized_entries:
        results['message'] = "No log entries could be normalized. Check log format." # Original message was a print
        return results
    print(f"Normalized {len(normalized_entries)} log entries.")

    # 4. Initialize AnomalyDetector and load custom rules
    anomaly_rules = None
    if anomaly_rules_file:
        try:
            with open(anomaly_rules_file, 'r', encoding='utf-8') as f:
                anomaly_rules = json.load(f)
            print(f"Loaded anomaly rules from {anomaly_rules_file}")
        except FileNotFoundError:
            print(f"Warning: Anomaly rules file not found at {anomaly_rules_file}. Using default rules.")
        except json.JSONDecodeError:
            print(f"Warning: Invalid JSON in anomaly rules file {anomaly_rules_file}. Using default rules.")
        except Exception as e:
            print(f"Warning: Error loading anomaly rules file: {e}. Using default rules.")

    anomaly_detector = AnomalyDetector(anomaly_rules=anomaly_rules)

    # 5. Detect anomalies
    detected_anomalies = anomaly_detector.detect_anomalies(normalized_entries)
    print(f"Detected {len(detected_anomalies)} anomalies.")

    # 6. Initialize IncidentCorrelator
    correlator_rules = {'min_events_for_incident': min_events}
    incident_correlator = IncidentCorrelator(time_window_seconds=time_window, correlation_rules=correlator_rules)

    # 7. Correlate incidents
    incidents = incident_correlator.correlate_incidents(normalized_entries)
    print(f"Correlated {len(incidents)} incidents.")

    # 8. Generate reports
    report_generator = ReportGenerator()

    # Use a unique name for reports based on the input file, to avoid overwriting
    base_filename_safe = os.path.basename(input_file_path).replace('.', '_').replace(' ', '_')

    csv_report_path = os.path.join(REPORTS_FOLDER, base_filename_safe + '_report.csv')
    report_generator.generate_csv_report(incidents, csv_report_path)
    if os.path.exists(csv_report_path):
        results['csv_report'] = csv_report_path
        print(f"CSV report generated: {csv_report_path}")

    json_report_path = os.path.join(REPORTS_FOLDER, base_filename_safe + '_report.json')
    report_generator.generate_json_report(incidents, json_report_path)
    if os.path.exists(json_report_path):
        results['json_report'] = json_report_path
        print(f"JSON report generated: {json_report_path}")

    # 9. Generate visualizations
    visualization_generator = VisualizationGenerator()

    timeline_plot_path = os.path.join(VISUALIZATIONS_FOLDER, base_filename_safe + '_timeline.png')
    visualization_generator.plot_incident_timeline(incidents, timeline_plot_path)
    if os.path.exists(timeline_plot_path):
        results['incident_timeline_plot'] = timeline_plot_path
        print(f"Incident timeline plot generated: {timeline_plot_path}")

    trends_plot_path = os.path.join(VISUALIZATIONS_FOLDER, base_filename_safe + '_trends.png')
    visualization_generator.plot_error_trends(normalized_entries, file_path=trends_plot_path)
    if os.path.exists(trends_plot_path):
        results['error_trends_plot'] = trends_plot_path
        print(f"Error trends plot generated: {trends_plot_path}")

    print("Analysis pipeline finished.")
    return results

# Define the new logic for the analyze_logs function
def new_analyze_logs():
    data = request.get_json()
    if not data or 'input_file' not in data:
        return jsonify({'error': 'Missing input_file in request body'}), 400

    input_filename = data['input_file']
    input_file_path = os.path.join(UPLOAD_FOLDER, input_filename) # Assume file is already in UPLOAD_FOLDER

    # Check if the uploaded file exists
    if not os.path.exists(input_file_path):
        return jsonify({'error': f'Uploaded file not found: {input_filename}'}), 404

    # Extract optional parameters with defaults
    time_window = data.get('time_window', 60)
    min_events = data.get('min_events', 1)
    log_format_regex = data.get('log_format_regex')
    log_format_mapping = data.get('log_format_mapping')
    log_format_timestamp = data.get('log_format_timestamp')
    anomaly_rules_file = data.get('anomaly_rules_file')

    # Run the analysis pipeline
    analysis_results = run_analysis_pipeline(
        input_file_path=input_file_path,
        time_window=time_window,
        min_events=min_events,
        log_format_regex=log_format_regex,
        log_format_mapping=log_format_mapping,
        log_format_timestamp=log_format_timestamp,
        anomaly_rules_file=anomaly_rules_file
    )

    # Check if analysis had an error message
    if 'Error' in analysis_results.get('message', '') and analysis_results.get('message') != 'Analysis completed successfully.':
        return jsonify(analysis_results), 500

    return jsonify({
        'message': 'Log analysis completed.',
        'results': analysis_results
    }), 200

# --- Explicitly update the existing /analyze API endpoint's view function ---
# This avoids the AssertionError by replacing the function without redefining the route
app.view_functions['analyze_logs'] = new_analyze_logs

print("Log analysis components integrated and /analyze endpoint updated.")

Log analysis components integrated and /analyze endpoint updated.


## Create Basic Web Interface

### Subtask:
Develop simple HTML templates or a basic frontend to allow users to interact with the web application. This interface should enable users to upload log files, view analysis results, and download reports/visualizations.


**Reasoning**:
To develop the web interface, I need to first create a 'templates' directory where Flask will look for HTML files, and then create an 'index.html' file within it, populating it with the required forms and placeholders for user interaction.



In [7]:
import os

# Define the directory for templates
TEMPLATES_FOLDER = 'templates'

# Create the templates directory if it doesn't exist
os.makedirs(TEMPLATES_FOLDER, exist_ok=True)

# Define the HTML content for index.html
index_html_content = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Log Analysis & Incident Correlation</title>
    <style>
        body { font-family: Arial, sans-serif; margin: 20px; background-color: #f4f4f4; color: #333; }
        .container { max-width: 900px; margin: auto; background: #fff; padding: 20px; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); }
        h1, h2 { color: #0056b3; }
        form { margin-bottom: 20px; padding: 15px; border: 1px solid #ddd; border-radius: 5px; background-color: #f9f9f9; }
        input[type="file"], input[type="submit"], button { padding: 10px 15px; border-radius: 5px; border: 1px solid #ccc; cursor: pointer; }
        input[type="submit"] { background-color: #28a745; color: white; border-color: #28a745; }
        input[type="submit"]:hover { background-color: #218838; }
        button { background-color: #007bff; color: white; border-color: #007bff; margin-left: 10px; }
        button:hover { background-color: #0056b3; }
        .section { margin-top: 30px; padding-top: 20px; border-top: 1px solid #eee; }
        .result-link { display: block; margin-bottom: 5px; color: #007bff; text-decoration: none; }
        .result-link:hover { text-decoration: underline; }
        img { max-width: 100%; height: auto; margin-top: 10px; border: 1px solid #ddd; border-radius: 5px; }
        .message { padding: 10px; margin-bottom: 15px; border-radius: 5px; }
        .success { background-color: #d4edda; color: #155724; border-color: #c3e6cb; }
        .error { background-color: #f8d7da; color: #721c24; border-color: #f5c6cb; }
    </style>
</head>
<body>
    <div class="container">
        <h1>Log Analysis and Incident Correlation System</h1>

        <div id="message-area" class="message" style="display:none;"></div>

        <form id="uploadForm" action="/upload" method="post" enctype="multipart/form-data">
            <h2>1. Upload Log File</h2>
            <input type="file" name="file" id="logFile" required>
            <input type="submit" value="Upload Log File">
        </form>

        <div class="section">
            <h2>2. Trigger Analysis</h2>
            <form id="analysisForm">
                <p>Uploaded File: <span id="uploadedFileName">None</span></p>
                <input type="hidden" id="inputFilePath" name="input_file">
                <label for="timeWindow">Correlation Time Window (seconds):</label>
                <input type="number" id="timeWindow" name="time_window" value="60" min="1" required>
                <br><br>
                <label for="minEvents">Minimum Events for Incident:</label>
                <input type="number" id="minEvents" name="min_events" value="1" min="1" required>
                <br><br>
                <label for="logFormatRegex">Custom Log Format Regex (optional):</label>
                <input type="text" id="logFormatRegex" name="log_format_regex" size="50"><br><br>
                <label for="logFormatMapping">Custom Log Format Mapping (JSON, optional):</label>
                <input type="text" id="logFormatMapping" name="log_format_mapping" size="50"><br><br>
                <label for="logFormatTimestamp">Custom Log Format Timestamp (e.g., %Y-%m-%d %H:%M:%S, optional):</label>
                <input type="text" id="logFormatTimestamp" name="log_format_timestamp" size="50"><br><br>
                <label for="anomalyRulesFile">Anomaly Rules File (e.g., custom_rules.json, optional):</label>
                <input type="text" id="anomalyRulesFile" name="anomaly_rules_file" size="50"><br><br>
                <button type="submit" id="analyzeButton" disabled>Analyze Logs</button>
            </form>
        </div>

        <div class="section">
            <h2>3. Analysis Results</h2>
            <div id="resultsArea">
                <p>Reports:</p>
                <a id="csvReportLink" class="result-link" style="display:none;" download>Download CSV Report</a>
                <a id="jsonReportLink" class="result-link" style="display:none;" download>Download JSON Report</a>

                <p>Visualizations:</p>
                <a id="timelineVizLink" class="result-link" style="display:none;" download>Download Incident Timeline Plot</a>
                <img id="timelineVizImage" style="display:none;" alt="Incident Timeline">
                <a id="trendsVizLink" class="result-link" style="display:none;" download>Download Error Trends Plot</a>
                <img id="trendsVizImage" style="display:none;" alt="Error Trends">
            </div>
        </div>

        <script>
            const uploadForm = document.getElementById('uploadForm');
            const analysisForm = document.getElementById('analysisForm');
            const logFile = document.getElementById('logFile');
            const uploadedFileName = document.getElementById('uploadedFileName');
            const inputFilePath = document.getElementById('inputFilePath');
            const analyzeButton = document.getElementById('analyzeButton');
            const resultsArea = document.getElementById('resultsArea');
            const messageArea = document.getElementById('message-area');

            uploadForm.addEventListener('submit', async (e) => {
                e.preventDefault();
                messageArea.style.display = 'none';
                const formData = new FormData(uploadForm);
                try {
                    const response = await fetch('/upload', {
                        method: 'POST',
                        body: formData
                    });
                    const result = await response.json();
                    if (response.ok) {
                        displayMessage(result.message, 'success');
                        uploadedFileName.textContent = logFile.files[0].name;
                        inputFilePath.value = logFile.files[0].name; // Store filename for analysis
                        analyzeButton.disabled = false;
                        clearResults();
                    } else {
                        displayMessage(result.error || 'Upload failed', 'error');
                    }
                } catch (error) {
                    displayMessage('Network error or server unreachable during upload.', 'error');
                    console.error('Upload Error:', error);
                }
            });

            analysisForm.addEventListener('submit', async (e) => {
                e.preventDefault();
                messageArea.style.display = 'none';
                clearResults();

                const input_file = inputFilePath.value;
                if (!input_file) {
                    displayMessage('Please upload a log file first.', 'error');
                    return;
                }

                const analysisData = {
                    input_file: input_file,
                    time_window: parseInt(document.getElementById('timeWindow').value),
                    min_events: parseInt(document.getElementById('minEvents').value),
                    log_format_regex: document.getElementById('logFormatRegex').value || null,
                    log_format_mapping: document.getElementById('logFormatMapping').value || null,
                    log_format_timestamp: document.getElementById('logFormatTimestamp').value || null,
                    anomaly_rules_file: document.getElementById('anomalyRulesFile').value || null
                };

                try {
                    const response = await fetch('/analyze', {
                        method: 'POST',
                        headers: {
                            'Content-Type': 'application/json'
                        },
                        body: JSON.stringify(analysisData)
                    });
                    const result = await response.json();

                    if (response.ok) {
                        displayMessage(result.message, 'success');
                        displayResults(result.results);
                    } else {
                        displayMessage(result.message || result.error || 'Analysis failed', 'error');
                        console.error('Analysis Error:', result);
                    }
                } catch (error) {
                    displayMessage('Network error or server unreachable during analysis.', 'error');
                    console.error('Analysis Error:', error);
                }
            });

            function displayMessage(message, type) {
                messageArea.textContent = message;
                messageArea.className = `message ${type}`;
                messageArea.style.display = 'block';
            }

            function clearResults() {
                document.getElementById('csvReportLink').style.display = 'none';
                document.getElementById('jsonReportLink').style.display = 'none';
                document.getElementById('timelineVizLink').style.display = 'none';
                document.getElementById('timelineVizImage').style.display = 'none';
                document.getElementById('trendsVizLink').style.display = 'none';
                document.getElementById('trendsVizImage').style.display = 'none';
            }

            function displayResults(results) {
                if (results.csv_report) {
                    const link = document.getElementById('csvReportLink');
                    link.href = '/reports/csv'; // Endpoint to download
                    link.style.display = 'block';
                }
                if (results.json_report) {
                    const link = document.getElementById('jsonReportLink');
                    link.href = '/reports/json'; // Endpoint to download
                    link.style.display = 'block';
                }
                if (results.incident_timeline_plot) {
                    const link = document.getElementById('timelineVizLink');
                    link.href = '/visualizations/timeline'; // Endpoint to download
                    link.style.display = 'block';
                    const img = document.getElementById('timelineVizImage');
                    img.src = '/visualizations/timeline?' + new Date().getTime(); // Add timestamp to bust cache
                    img.style.display = 'block';
                }
                if (results.error_trends_plot) {
                    const link = document.getElementById('trendsVizLink');
                    link.href = '/visualizations/trends'; // Endpoint to download
                    link.style.display = 'block';
                    const img = document.getElementById('trendsVizImage');
                    img.src = '/visualizations/trends?' + new Date().getTime(); // Add timestamp to bust cache
                    img.style.display = 'block';
                }
            }
        </script>
    </div>
</body>
</html>
"""

# Write the HTML content to index.html inside the templates directory
index_html_file_path = os.path.join(TEMPLATES_FOLDER, 'index.html')
with open(index_html_file_path, 'w', encoding='utf-8') as f:
    f.write(index_html_content)

print(f"Created '{TEMPLATES_FOLDER}' directory and 'index.html' at '{index_html_file_path}'.")

Created 'templates' directory and 'index.html' at 'templates/index.html'.


**Reasoning**:
With the `index.html` file created, I need to modify the Flask application to render this HTML file when a user accesses the root URL (`/`). This will make the web interface visible and interactive.



In [8]:
from flask import Flask, render_template

# Assuming 'app' is already defined from a previous cell
# If starting fresh, uncomment: app = Flask(__name__)

# Reconfigure app to look for templates in the 'templates' folder
app.template_folder = 'templates'

# Modify the existing '/' route to render index.html
@app.route('/')
def index():
    return render_template('index.html')

print("Flask app's '/' route updated to render index.html.")
print("To run the Flask app from a Python file (e.g., app.py), you would typically use:")
print("export FLASK_APP=app.py")
print("flask run")
print("In a Colab environment, you would need to set up a public URL service (like ngrok) to access it from a browser.")

Flask app's '/' route updated to render index.html.
To run the Flask app from a Python file (e.g., app.py), you would typically use:
export FLASK_APP=app.py
flask run
In a Colab environment, you would need to set up a public URL service (like ngrok) to access it from a browser.


## Serve Static Files and Templates

### Subtask:
Configure the Flask application to properly serve static files (like CSS) and ensure HTML templates are rendered correctly.


**Reasoning**:
First, I need to create the `static` directory and extract the inline CSS from the `index_html_content` variable to create a `style.css` file within it, as specified in the instructions.



In [9]:
import os

# Define the directory for static files
STATIC_FOLDER = 'static'

# Create the static directory if it doesn't exist
os.makedirs(STATIC_FOLDER, exist_ok=True)
print(f"Created '{STATIC_FOLDER}' directory.")

# Extract CSS from the previously defined index_html_content
# The CSS is between <style> and </style> tags
# Using a regex for a robust extraction
import re

# The original index_html_content is available from previous steps
# For this cell, we'll assume it's still in scope or re-fetch it if necessary.
# For safety, let's re-define it as it might not persist across sessions
index_html_content_old = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Log Analysis & Incident Correlation</title>
    <style>
        body { font-family: Arial, sans-serif; margin: 20px; background-color: #f4f4f4; color: #333; }
        .container { max-width: 900px; margin: auto; background: #fff; padding: 20px; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); }
        h1, h2 { color: #0056b3; }
        form { margin-bottom: 20px; padding: 15px; border: 1px solid #ddd; border-radius: 5px; background-color: #f9f9f9; }
        input[type="file"], input[type="submit"], button { padding: 10px 15px; border-radius: 5px; border: 1px solid #ccc; cursor: pointer; }
        input[type="submit"] { background-color: #28a745; color: white; border-color: #28a745; }
        input[type="submit"]:hover { background-color: #218838; }
        button { background-color: #007bff; color: white; border-color: #007bff; margin-left: 10px; }
        button:hover { background-color: #0056b3; }
        .section { margin-top: 30px; padding-top: 20px; border-top: 1px solid #eee; }
        .result-link { display: block; margin-bottom: 5px; color: #007bff; text-decoration: none; }
        .result-link:hover { text-decoration: underline; }
        img { max-width: 100%; height: auto; margin-top: 10px; border: 1px solid #ddd; border-radius: 5px; }
        .message { padding: 10px; margin-bottom: 15px; border-radius: 5px; }
        .success { background-color: #d4edda; color: #155724; border-color: #c3e6cb; }
        .error { background-color: #f8d7da; color: #721c24; border-color: #f5c6cb; }
    </style>
</head>
<body>
    <div class="container">
        <h1>Log Analysis and Incident Correlation System</h1>

        <div id="message-area" class="message" style="display:none;"></div>

        <form id="uploadForm" action="/upload" method="post" enctype="multipart/form-data">
            <h2>1. Upload Log File</h2>
            <input type="file" name="file" id="logFile" required>
            <input type="submit" value="Upload Log File">
        </form>

        <div class="section">
            <h2>2. Trigger Analysis</h2>
            <form id="analysisForm">
                <p>Uploaded File: <span id="uploadedFileName">None</span></p>
                <input type="hidden" id="inputFilePath" name="input_file">
                <label for="timeWindow">Correlation Time Window (seconds):</label>
                <input type="number" id="timeWindow" name="time_window" value="60" min="1" required>
                <br><br>
                <label for="minEvents">Minimum Events for Incident:</label>
                <input type="number" id="minEvents" name="min_events" value="1" min="1" required>
                <br><br>
                <label for="logFormatRegex">Custom Log Format Regex (optional):</label>
                <input type="text" id="logFormatRegex" name="log_format_regex" size="50"><br><br>
                <label for="logFormatMapping">Custom Log Format Mapping (JSON, optional):</label>
                <input type="text" id="logFormatMapping" name="log_format_mapping" size="50"><br><br>
                <label for="logFormatTimestamp">Custom Log Format Timestamp (e.g., %Y-%m-%d %H:%M:%S, optional):</label>
                <input type="text" id="logFormatTimestamp" name="log_format_timestamp" size="50"><br><br>
                <label for="anomalyRulesFile">Anomaly Rules File (e.g., custom_rules.json, optional):</label>
                <input type="text" id="anomalyRulesFile" name="anomaly_rules_file" size="50"><br><br>
                <button type="submit" id="analyzeButton" disabled>Analyze Logs</button>
            </form>
        </div>

        <div class="section">
            <h2>3. Analysis Results</h2>
            <div id="resultsArea">
                <p>Reports:</p>
                <a id="csvReportLink" class="result-link" style="display:none;" download>Download CSV Report</a>
                <a id="jsonReportLink" class="result-link" style="display:none;" download>Download JSON Report</a>

                <p>Visualizations:</p>
                <a id="timelineVizLink" class="result-link" style="display:none;" download>Download Incident Timeline Plot</a>
                <img id="timelineVizImage" style="display:none;" alt="Incident Timeline">
                <a id="trendsVizLink" class="result-link" style="display:none;" download>Download Error Trends Plot</a>
                <img id="trendsVizImage" style="display:none;" alt="Error Trends">
            </div>
        </div>

        <script>
            const uploadForm = document.getElementById('uploadForm');
            const analysisForm = document.getElementById('analysisForm');
            const logFile = document.getElementById('logFile');
            const uploadedFileName = document.getElementById('uploadedFileName');
            const inputFilePath = document.getElementById('inputFilePath');
            const analyzeButton = document.getElementById('analyzeButton');
            const resultsArea = document.getElementById('resultsArea');
            const messageArea = document.getElementById('message-area');

            uploadForm.addEventListener('submit', async (e) => {
                e.preventDefault();
                messageArea.style.display = 'none';
                const formData = new FormData(uploadForm);
                try {
                    const response = await fetch('/upload', {
                        method: 'POST',
                        body: formData
                    });
                    const result = await response.json();
                    if (response.ok) {
                        displayMessage(result.message, 'success');
                        uploadedFileName.textContent = logFile.files[0].name;
                        inputFilePath.value = logFile.files[0].name; // Store filename for analysis
                        analyzeButton.disabled = false;
                        clearResults();
                    } else {
                        displayMessage(result.error || 'Upload failed', 'error');
                    }
                } catch (error) {
                    displayMessage('Network error or server unreachable during upload.', 'error');
                    console.error('Upload Error:', error);
                }
            });

            analysisForm.addEventListener('submit', async (e) => {
                e.preventDefault();
                messageArea.style.display = 'none';
                clearResults();

                const input_file = inputFilePath.value;
                if (!input_file) {
                    displayMessage('Please upload a log file first.', 'error');
                    return;
                }

                const analysisData = {
                    input_file: input_file,
                    time_window: parseInt(document.getElementById('timeWindow').value),
                    min_events: parseInt(document.getElementById('minEvents').value),
                    log_format_regex: document.getElementById('logFormatRegex').value || null,
                    log_format_mapping: document.getElementById('logFormatMapping').value || null,
                    log_format_timestamp: document.getElementById('logFormatTimestamp').value || null,
                    anomaly_rules_file: document.getElementById('anomalyRulesFile').value || null
                };

                try {
                    const response = await fetch('/analyze', {
                        method: 'POST',
                        headers: {
                            'Content-Type': 'application/json'
                        },
                        body: JSON.stringify(analysisData)
                    });
                    const result = await response.json();

                    if (response.ok) {
                        displayMessage(result.message, 'success');
                        displayResults(result.results);
                    } else {
                        displayMessage(result.message || result.error || 'Analysis failed', 'error');
                        console.error('Analysis Error:', result);
                    }
                } catch (error) {
                    displayMessage('Network error or server unreachable during analysis.', 'error');
                    console.error('Analysis Error:', error);
                }
            });

            function displayMessage(message, type) {
                messageArea.textContent = message;
                messageArea.className = `message ${type}`;
                messageArea.style.display = 'block';
            }

            function clearResults() {
                document.getElementById('csvReportLink').style.display = 'none';
                document.getElementById('jsonReportLink').style.display = 'none';
                document.getElementById('timelineVizLink').style.display = 'none';
                document.getElementById('timelineVizImage').style.display = 'none';
                document.getElementById('trendsVizLink').style.display = 'none';
                document.getElementById('trendsVizImage').style.display = 'none';
            }

            function displayResults(results) {
                if (results.csv_report) {
                    const link = document.getElementById('csvReportLink');
                    link.href = '/reports/csv'; // Endpoint to download
                    link.style.display = 'block';
                }
                if (results.json_report) {
                    const link = document.getElementById('jsonReportLink');
                    link.href = '/reports/json'; // Endpoint to download
                    link.style.display = 'block';
                }
                if (results.incident_timeline_plot) {
                    const link = document.getElementById('timelineVizLink');
                    link.href = '/visualizations/timeline'; // Endpoint to download
                    link.style.display = 'block';
                    const img = document.getElementById('timelineVizImage');
                    img.src = '/visualizations/timeline?' + new Date().getTime(); // Add timestamp to bust cache
                    img.style.display = 'block';
                }
                if (results.error_trends_plot) {
                    const link = document.getElementById('trendsVizLink');
                    link.href = '/visualizations/trends'; // Endpoint to download
                    link.style.display = 'block';
                    const img = document.getElementById('trendsVizImage');
                    img.src = '/visualizations/trends?' + new Date().getTime(); // Add timestamp to bust cache
                    img.style.display = 'block';
                }
            }
        </script>
    </div>
</body>
</html>
"""

css_match = re.search(r'<style>(.*?)</style>', index_html_content_old, re.DOTALL)
css_content = ''
if css_match:
    css_content = css_match.group(1).strip()

# Write the CSS content to style.css inside the static directory
style_css_file_path = os.path.join(STATIC_FOLDER, 'style.css')
with open(style_css_file_path, 'w', encoding='utf-8') as f:
    f.write(css_content)

print(f"Extracted CSS and saved to '{style_css_file_path}'.")

Created 'static' directory.
Extracted CSS and saved to 'static/style.css'.


**Reasoning**:
Now that the CSS has been extracted, I need to update the `index_html_content` to reference the external `style.css` file using a `<link>` tag and then overwrite the `index.html` file in the `templates` directory with this updated content. This ensures Flask serves static files correctly.



In [10]:
import os

# Define the directory for templates and static files (already defined in previous cells but for context)
TEMPLATES_FOLDER = 'templates'
STATIC_FOLDER = 'static'

# Create the templates directory if it doesn't exist (already done)
os.makedirs(TEMPLATES_FOLDER, exist_ok=True)

# Updated HTML content for index.html, replacing inline style with a link to style.css
index_html_content_updated = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Log Analysis & Incident Correlation</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
    <div class="container">
        <h1>Log Analysis and Incident Correlation System</h1>

        <div id="message-area" class="message" style="display:none;"></div>

        <form id="uploadForm" action="/upload" method="post" enctype="multipart/form-data">
            <h2>1. Upload Log File</h2>
            <input type="file" name="file" id="logFile" required>
            <input type="submit" value="Upload Log File">
        </form>

        <div class="section">
            <h2>2. Trigger Analysis</h2>
            <form id="analysisForm">
                <p>Uploaded File: <span id="uploadedFileName">None</span></p>
                <input type="hidden" id="inputFilePath" name="input_file">
                <label for="timeWindow">Correlation Time Window (seconds):</label>
                <input type="number" id="timeWindow" name="time_window" value="60" min="1" required>
                <br><br>
                <label for="minEvents">Minimum Events for Incident:</label>
                <input type="number" id="minEvents" name="min_events" value="1" min="1" required>
                <br><br>
                <label for="logFormatRegex">Custom Log Format Regex (optional):</label>
                <input type="text" id="logFormatRegex" name="log_format_regex" size="50"><br><br>
                <label for="logFormatMapping">Custom Log Format Mapping (JSON, optional):</label>
                <input type="text" id="logFormatMapping" name="log_format_mapping" size="50"><br><br>
                <label for="logFormatTimestamp">Custom Log Format Timestamp (e.g., %Y-%m-%d %H:%M:%S, optional):</label>
                <input type="text" id="logFormatTimestamp" name="log_format_timestamp" size="50"><br><br>
                <label for="anomalyRulesFile">Anomaly Rules File (e.g., custom_rules.json, optional):</label>
                <input type="text" id="anomalyRulesFile" name="anomaly_rules_file" size="50"><br><br>
                <button type="submit" id="analyzeButton" disabled>Analyze Logs</button>
            </form>
        </div>

        <div class="section">
            <h2>3. Analysis Results</h2>
            <div id="resultsArea">
                <p>Reports:</p>
                <a id="csvReportLink" class="result-link" style="display:none;" download>Download CSV Report</a>
                <a id="jsonReportLink" class="result-link" style="display:none;" download>Download JSON Report</a>

                <p>Visualizations:</p>
                <a id="timelineVizLink" class="result-link" style="display:none;" download>Download Incident Timeline Plot</a>
                <img id="timelineVizImage" style="display:none;" alt="Incident Timeline">
                <a id="trendsVizLink" class="result-link" style="display:none;" download>Download Error Trends Plot</a>
                <img id="trendsVizImage" style="display:none;" alt="Error Trends">
            </div>
        </div>

        <script>
            const uploadForm = document.getElementById('uploadForm');
            const analysisForm = document.getElementById('analysisForm');
            const logFile = document.getElementById('logFile');
            const uploadedFileName = document.getElementById('uploadedFileName');
            const inputFilePath = document.getElementById('inputFilePath');
            const analyzeButton = document.getElementById('analyzeButton');
            const resultsArea = document.getElementById('resultsArea');
            const messageArea = document.getElementById('message-area');

            uploadForm.addEventListener('submit', async (e) => {
                e.preventDefault();
                messageArea.style.display = 'none';
                const formData = new FormData(uploadForm);
                try {
                    const response = await fetch('/upload', {
                        method: 'POST',
                        body: formData
                    });
                    const result = await response.json();
                    if (response.ok) {
                        displayMessage(result.message, 'success');
                        uploadedFileName.textContent = logFile.files[0].name;
                        inputFilePath.value = logFile.files[0].name; // Store filename for analysis
                        analyzeButton.disabled = false;
                        clearResults();
                    } else {
                        displayMessage(result.error || 'Upload failed', 'error');
                    }
                } catch (error) {
                    displayMessage('Network error or server unreachable during upload.', 'error');
                    console.error('Upload Error:', error);
                }
            });

            analysisForm.addEventListener('submit', async (e) => {
                e.preventDefault();
                messageArea.style.display = 'none';
                clearResults();

                const input_file = inputFilePath.value;
                if (!input_file) {
                    displayMessage('Please upload a log file first.', 'error');
                    return;
                }

                const analysisData = {
                    input_file: input_file,
                    time_window: parseInt(document.getElementById('timeWindow').value),
                    min_events: parseInt(document.getElementById('minEvents').value),
                    log_format_regex: document.getElementById('logFormatRegex').value || null,
                    log_format_mapping: document.getElementById('logFormatMapping').value || null,
                    log_format_timestamp: document.getElementById('logFormatTimestamp').value || null,
                    anomaly_rules_file: document.getElementById('anomalyRulesFile').value || null
                };

                try {
                    const response = await fetch('/analyze', {
                        method: 'POST',
                        headers: {
                            'Content-Type': 'application/json'
                        },
                        body: JSON.stringify(analysisData)
                    });
                    const result = await response.json();

                    if (response.ok) {
                        displayMessage(result.message, 'success');
                        displayResults(result.results);
                    } else {
                        displayMessage(result.message || result.error || 'Analysis failed', 'error');
                        console.error('Analysis Error:', result);
                    }
                } catch (error) {
                    displayMessage('Network error or server unreachable during analysis.', 'error');
                    console.error('Analysis Error:', error);
                }
            });

            function displayMessage(message, type) {
                messageArea.textContent = message;
                messageArea.className = `message ${type}`;
                messageArea.style.display = 'block';
            }

            function clearResults() {
                document.getElementById('csvReportLink').style.display = 'none';
                document.getElementById('jsonReportLink').style.display = 'none';
                document.getElementById('timelineVizLink').style.display = 'none';
                document.getElementById('timelineVizImage').style.display = 'none';
                document.getElementById('trendsVizLink').style.display = 'none';
                document.getElementById('trendsVizImage').style.display = 'none';
            }

            function displayResults(results) {
                if (results.csv_report) {
                    const link = document.getElementById('csvReportLink');
                    link.href = '/reports/csv'; // Endpoint to download
                    link.style.display = 'block';
                }
                if (results.json_report) {
                    const link = document.getElementById('jsonReportLink');
                    link.href = '/reports/json'; // Endpoint to download
                    link.style.display = 'block';
                }
                if (results.incident_timeline_plot) {
                    const link = document.getElementById('timelineVizLink');
                    link.href = '/visualizations/timeline'; // Endpoint to download
                    link.style.display = 'block';
                    const img = document.getElementById('timelineVizImage');
                    img.src = '/visualizations/timeline?' + new Date().getTime(); // Add timestamp to bust cache
                    img.style.display = 'block';
                }
                if (results.error_trends_plot) {
                    const link = document.getElementById('trendsVizLink');
                    link.href = '/visualizations/trends'; // Endpoint to download
                    link.style.display = 'block';
                    const img = document.getElementById('trendsVizImage');
                    img.src = '/visualizations/trends?' + new Date().getTime(); // Add timestamp to bust cache
                    img.style.display = 'block';
                }
            }
        </script>
    </div>
</body>
</html>
"""

# Write the updated HTML content to index.html inside the templates directory
index_html_file_path = os.path.join(TEMPLATES_FOLDER, 'index.html')
with open(index_html_file_path, 'w', encoding='utf-8') as f:
    f.write(index_html_content_updated)

print(f"Updated 'index.html' at '{index_html_file_path}' to link to 'static/style.css'.")

SyntaxError: f-string: expecting '=', or '!', or ':', or '}' (ipython-input-3644713176.py, line 81)

**Reasoning**:
The previous code failed with a `SyntaxError` because the f-string syntax `f"""..."""` was interpreting the literal curly braces within the embedded JavaScript code as f-string placeholders. To fix this, I will change `index_html_content_updated` to a regular triple-quoted string and insert the `url_for` line as a plain string, avoiding f-string interpolation for the entire HTML block.



In [11]:
import os

# Define the directory for templates and static files (already defined in previous cells but for context)
TEMPLATES_FOLDER = 'templates'
STATIC_FOLDER = 'static'

# Create the templates directory if it doesn't exist (already done)
os.makedirs(TEMPLATES_FOLDER, exist_ok=True)

# Updated HTML content for index.html, replacing inline style with a link to style.css
index_html_content_updated = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Log Analysis & Incident Correlation</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
    <div class="container">
        <h1>Log Analysis and Incident Correlation System</h1>

        <div id="message-area" class="message" style="display:none;"></div>

        <form id="uploadForm" action="/upload" method="post" enctype="multipart/form-data">
            <h2>1. Upload Log File</h2>
            <input type="file" name="file" id="logFile" required>
            <input type="submit" value="Upload Log File">
        </form>

        <div class="section">
            <h2>2. Trigger Analysis</h2>
            <form id="analysisForm">
                <p>Uploaded File: <span id="uploadedFileName">None</span></p>
                <input type="hidden" id="inputFilePath" name="input_file">
                <label for="timeWindow">Correlation Time Window (seconds):</label>
                <input type="number" id="timeWindow" name="time_window" value="60" min="1" required>
                <br><br>
                <label for="minEvents">Minimum Events for Incident:</label>
                <input type="number" id="minEvents" name="min_events" value="1" min="1" required>
                <br><br>
                <label for="logFormatRegex">Custom Log Format Regex (optional):</label>
                <input type="text" id="logFormatRegex" name="log_format_regex" size="50"><br><br>
                <label for="logFormatMapping">Custom Log Format Mapping (JSON, optional):</label>
                <input type="text" id="logFormatMapping" name="log_format_mapping" size="50"><br><br>
                <label for="logFormatTimestamp">Custom Log Format Timestamp (e.g., %Y-%m-%d %H:%M:%S, optional):</label>
                <input type="text" id="logFormatTimestamp" name="log_format_timestamp" size="50"><br><br>
                <label for="anomalyRulesFile">Anomaly Rules File (e.g., custom_rules.json, optional):</label>
                <input type="text" id="anomalyRulesFile" name="anomaly_rules_file" size="50"><br><br>
                <button type="submit" id="analyzeButton" disabled>Analyze Logs</button>
            </form>
        </div>

        <div class="section">
            <h2>3. Analysis Results</h2>
            <div id="resultsArea">
                <p>Reports:</p>
                <a id="csvReportLink" class="result-link" style="display:none;" download>Download CSV Report</a>
                <a id="jsonReportLink" class="result-link" style="display:none;" download>Download JSON Report</a>

                <p>Visualizations:</p>
                <a id="timelineVizLink" class="result-link" style="display:none;" download>Download Incident Timeline Plot</a>
                <img id="timelineVizImage" style="display:none;" alt="Incident Timeline">
                <a id="trendsVizLink" class="result-link" style="display:none;" download>Download Error Trends Plot</a>
                <img id="trendsVizImage" style="display:none;" alt="Error Trends">
            </div>
        </div>

        <script>
            const uploadForm = document.getElementById('uploadForm');
            const analysisForm = document.getElementById('analysisForm');
            const logFile = document.getElementById('logFile');
            const uploadedFileName = document.getElementById('uploadedFileName');
            const inputFilePath = document.getElementById('inputFilePath');
            const analyzeButton = document.getElementById('analyzeButton');
            const resultsArea = document.getElementById('resultsArea');
            const messageArea = document.getElementById('message-area');

            uploadForm.addEventListener('submit', async (e) => {
                e.preventDefault();
                messageArea.style.display = 'none';
                const formData = new FormData(uploadForm);
                try {
                    const response = await fetch('/upload', {
                        method: 'POST',
                        body: formData
                    });
                    const result = await response.json();
                    if (response.ok) {
                        displayMessage(result.message, 'success');
                        uploadedFileName.textContent = logFile.files[0].name;
                        inputFilePath.value = logFile.files[0].name; // Store filename for analysis
                        analyzeButton.disabled = false;
                        clearResults();
                    } else {
                        displayMessage(result.error || 'Upload failed', 'error');
                    }
                } catch (error) {
                    displayMessage('Network error or server unreachable during upload.', 'error');
                    console.error('Upload Error:', error);
                }
            });

            analysisForm.addEventListener('submit', async (e) => {
                e.preventDefault();
                messageArea.style.display = 'none';
                clearResults();

                const input_file = inputFilePath.value;
                if (!input_file) {
                    displayMessage('Please upload a log file first.', 'error');
                    return;
                }

                const analysisData = {
                    input_file: input_file,
                    time_window: parseInt(document.getElementById('timeWindow').value),
                    min_events: parseInt(document.getElementById('minEvents').value),
                    log_format_regex: document.getElementById('logFormatRegex').value || null,
                    log_format_mapping: document.getElementById('logFormatMapping').value || null,
                    log_format_timestamp: document.getElementById('logFormatTimestamp').value || null,
                    anomaly_rules_file: document.getElementById('anomalyRulesFile').value || null
                };

                try {
                    const response = await fetch('/analyze', {
                        method: 'POST',
                        headers: {
                            'Content-Type': 'application/json'
                        },
                        body: JSON.stringify(analysisData)
                    });
                    const result = await response.json();

                    if (response.ok) {
                        displayMessage(result.message, 'success');
                        displayResults(result.results);
                    } else {
                        displayMessage(result.message || result.error || 'Analysis failed', 'error');
                        console.error('Analysis Error:', result);
                    }
                } catch (error) {
                    displayMessage('Network error or server unreachable during analysis.', 'error');
                    console.error('Analysis Error:', error);
                }
            });

            function displayMessage(message, type) {
                messageArea.textContent = message;
                messageArea.className = `message ${type}`;
                messageArea.style.display = 'block';
            }

            function clearResults() {
                document.getElementById('csvReportLink').style.display = 'none';
                document.getElementById('jsonReportLink').style.display = 'none';
                document.getElementById('timelineVizLink').style.display = 'none';
                document.getElementById('timelineVizImage').style.display = 'none';
                document.getElementById('trendsVizLink').style.display = 'none';
                document.getElementById('trendsVizImage').style.display = 'none';
            }

            function displayResults(results) {
                if (results.csv_report) {
                    const link = document.getElementById('csvReportLink');
                    link.href = '/reports/csv'; // Endpoint to download
                    link.style.display = 'block';
                }
                if (results.json_report) {
                    const link = document.getElementById('jsonReportLink');
                    link.href = '/reports/json'; // Endpoint to download
                    link.style.display = 'block';
                }
                if (results.incident_timeline_plot) {
                    const link = document.getElementById('timelineVizLink');
                    link.href = '/visualizations/timeline'; // Endpoint to download
                    link.style.display = 'block';
                    const img = document.getElementById('timelineVizImage');
                    img.src = '/visualizations/timeline?' + new Date().getTime(); // Add timestamp to bust cache
                    img.style.display = 'block';
                }
                if (results.error_trends_plot) {
                    const link = document.getElementById('trendsVizLink');
                    link.href = '/visualizations/trends'; // Endpoint to download
                    link.style.display = 'block';
                    const img = document.getElementById('trendsVizImage');
                    img.src = '/visualizations/trends?' + new Date().getTime(); // Add timestamp to bust cache
                    img.style.display = 'block';
                }
            }
        </script>
    </div>
</body>
</html>
"""

# Write the updated HTML content to index.html inside the templates directory
index_html_file_path = os.path.join(TEMPLATES_FOLDER, 'index.html')
with open(index_html_file_path, 'w', encoding='utf-8') as f:
    f.write(index_html_content_updated)

print(f"Updated 'index.html' at '{index_html_file_path}' to link to 'static/style.css'.")

Updated 'index.html' at 'templates/index.html' to link to 'static/style.css'.


## Final Task

### Subtask:
Summarize the developed web application, including its functionalities, and provide instructions on how to run it locally or deploy it.


## Summary:

### Q&A

The developed web application is a **Log Analysis and Incident Correlation System** built using the Flask web framework. It provides a user-friendly web interface for end-to-end processing of log files, from upload to analysis and reporting.

Its primary functionalities include:
*   **Log File Upload**: Users can upload raw log files through a web form, which are then stored on the server.
*   **Configurable Log Analysis**: The system triggers a comprehensive analysis pipeline on uploaded log files. Users can specify parameters such as:
    *   **Correlation Time Window**: Defines the time frame for grouping related log events into incidents.
    *   **Minimum Events for Incident**: Sets the threshold for the number of events required to constitute a correlated incident.
    *   **Custom Log Format**: Supports defining custom regular expressions, mappings, and timestamp formats for parsing diverse log structures.
    *   **Anomaly Rules**: Allows for specifying a file containing custom rules to enhance anomaly detection.
*   **Report Generation and Download**: After analysis, the system generates structured incident reports, which can be downloaded in both CSV and JSON formats.
*   **Visualization Generation and Display**: The application creates visual summaries of the analysis, including an incident timeline plot and error trends plot. These visualizations are displayed directly on the web interface and can also be downloaded.
*   **Dynamic Feedback**: The frontend provides real-time feedback on upload and analysis status, including success/error messages.

To run this application locally or in a Colab environment:
1.  **Dependencies**: Ensure Flask and `matplotlib` (for visualizations) are installed. In a Python environment, this is typically done via `pip install Flask matplotlib`.
2.  **Application Structure**:
    *   An `app` instance of `Flask` needs to be initialized.
    *   Necessary directories (`uploads`, `reports`, `visualizations`, `templates`, `static`) must exist. The application code automatically creates these.
    *   The `templates/index.html` file (containing the frontend HTML and JavaScript) and `static/style.css` (for styling) must be in place. The development process populated these files.
3.  **Execution (Local)**:
    *   Save all the Python logic (Flask app definition, routes, analysis classes, pipeline function) into a Python file, e.g., `app.py`.
    *   Open a terminal in the directory containing `app.py`.
    *   Set the Flask application environment variable: `export FLASK_APP=app.py` (Linux/macOS) or `set FLASK_APP=app.py` (Windows).
    *   Run the Flask development server: `flask run`.
    *   Access the application in your web browser at the provided local address (e.g., `http://127.0.0.1:5000/`).
4.  **Execution (Colab)**:
    *   Due to the nature of Colab notebooks, direct external access requires a tunneling service. Tools like `ngrok` are commonly used to create a public URL for a local Flask server running within Colab.
    *   The Flask application code as developed would be executed within the Colab cells. An additional step to expose the Flask port via `ngrok` (or similar) would be required to access the web interface from outside the Colab environment.

### Data Analysis Key Findings

*   **Flask Web Framework Setup**: The Flask library was successfully installed and initialized. A basic `app` instance was created, and an initial `/` route was defined to confirm basic functionality.
*   **Robust API Endpoints Created**:
    *   Four primary API endpoints were established: `/upload` (POST), `/analyze` (POST), `/reports/<format>` (GET), and `/visualizations/<type>` (GET).
    *   Dedicated directories (`uploads`, `reports`, `visualizations`) were created to organize incoming log files, generated reports, and visualizations, respectively.
    *   The `/upload` endpoint handles file reception and saving, while `/analyze` is designed to trigger the full analysis pipeline using parameters from the request body.
    *   `/reports` and `/visualizations` endpoints serve generated files, ensuring proper error handling for invalid formats or missing files.
*   **Integrated Log Analysis Pipeline**:
    *   All core log analysis components (`LogParser`, `LogNormalizer`, `AnomalyDetector`, `IncidentCorrelator`, `ReportGenerator`, `VisualizationGenerator`) were integrated into a single, comprehensive `run_analysis_pipeline` function.
    *   This pipeline automates the entire workflow: reading raw logs, parsing using defined formats (including custom options), normalizing entries, detecting anomalies based on rules, correlating incidents within a specified time window, and finally generating CSV/JSON reports and PNG visualizations (incident timeline, error trends).
    *   The `/analyze` endpoint was updated to invoke this pipeline, accepting dynamic parameters for analysis customization.
*   **Interactive Web Interface Developed**:
    *   A `templates/index.html` file was created, serving as the application's frontend.
    *   The interface allows users to upload log files, configure analysis parameters (e.g., correlation time window, custom log format regex and mapping, anomaly rules file), and trigger the analysis.
    *   Frontend JavaScript handles asynchronous communication with the Flask backend, dynamically updating the UI with upload/analysis status messages and providing links to download generated reports and view visualizations.
*   **Static File and Template Serving**:
    *   The application was configured to properly serve static assets by creating a `static/` directory and extracting the inline CSS into `static/style.css`.
    *   The `index.html` template was updated to correctly link to this external stylesheet using Flask's `url_for` function, ensuring that the web interface is styled as intended.

### Insights or Next Steps

*   The integrated web application provides a complete, albeit basic, platform for log analysis. The modular design with distinct API endpoints and a centralized analysis pipeline allows for easy maintenance and potential future extensions.
*   For deployment in a production environment, enhancements would be needed, including user authentication, more robust input validation (especially for custom log formats and anomaly rules), asynchronous processing of analysis tasks to prevent UI blocking, and containerization (e.g., Docker) for easier deployment and scaling.
